In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
import mysql.connector
from google.cloud import storage
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

# EXTRACT

## Extract data from database back end

In [2]:
load_dotenv()

def extract_all_tables_to_csv(output_folder):
    try:
        with mysql.connector.connect(
            host=os.getenv("host"),
            port="3306",
            user="root",
            password=os.getenv("password"),
            database="recything_db"
        ) as conn:
            cursor = conn.cursor()
            cursor.execute("SHOW TABLES")
            tables = cursor.fetchall()
            for table in tables:
                table_name = table[0]
                query = f"SELECT * FROM {table_name}"
                df = pd.read_sql(query, conn)
                file_path = f"{output_folder}/{table_name}.csv"
                df.to_csv(file_path, index=False)
            print("Data extraction completed successfully")
    except mysql.connector.Error as err:
        print(f"Error: {err}")

extract_all_tables_to_csv("data_database")

C:\Users\yazid\AppData\Local\Temp\ipykernel_21044\279712126.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Data extraction completed successfully


### Table about_us_images

In [3]:
df_about_us_images = pd.read_csv("data_database/about_us_images.csv")
df_about_us_images.head()

,id,about_us_id,name,image_url,created_at,updated_at,deleted_at
0,ABSI01,ABS01,NaN,https://res.cloudinary.com/dlbbsdd3a/image/upl...,2024-06-15 13:03:33.756,2024-06-15 13:03:33.756,NaN
1,ABSI02,ABS05,NaN,https://res.cloudinary.com/dlbbsdd3a/image/upl...,2024-06-15 13:03:33.760,2024-06-15 13:03:33.760,NaN
2,ABSI03,ABS05,NaN,https://res.cloudinary.com/dlbbsdd3a/image/upl...,2024-06-15 13:03:33.766,2024-06-15 13:03:33.766,NaN
3,ABSI04,ABS06,Hadyan Alhafizh,https://res.cloudinary.com/dlbbsdd3a/image/upl...,2024-06-15 13:03:33.770,2024-06-15 13:03:33.770,NaN
4,ABSI05,ABS06,Leonita Puteri Kurniawan,https://res.cloudinary.com/dlbbsdd3a/image/upl...,2024-06-15 13:03:33.774,2024-06-15 13:03:33.774,NaN


### Table about_us

In [4]:
df_about_us = pd.read_csv("data_database/about_us.csv")
df_about_us.head()

,id,category,title,description,created_at,updated_at,deleted_at
0,ABS01,perusahaan,Tentang siapa kami,RecyThing adalah pemimpin di industri daur ula...,2024-06-15 13:03:33.682,2024-06-15 13:03:33.682,NaN
1,ABS02,perusahaan,Visi Kami,Menciptakan masyarakat yang sadar lingkungan d...,2024-06-15 13:03:33.687,2024-06-15 13:03:33.687,NaN
2,ABS03,perusahaan,Komitmen Kami,Prioritaskan penggunaan teknologi terbaru dan ...,2024-06-15 13:03:33.692,2024-06-15 13:03:33.692,NaN
3,ABS04,perusahaan,Pelayanan Pelanggan Unggul,Tim ahli yang berpengalaman memberikan solusi ...,2024-06-15 13:03:33.696,2024-06-15 13:03:33.696,NaN
4,ABS05,perusahaan,Pendidikan Masyarakat,Berperan aktif dalam mendidik masyarakat tenta...,2024-06-15 13:03:33.700,2024-06-15 13:03:33.700,NaN


### Table achievements

In [5]:
df_achievements = pd.read_csv("data_database/achievements.csv")
df_achievements.head()

,id,level,target_point,badge_url,badge_url_user,created_at,updated_at,deleted_at
0,1,classic,0,https://res.cloudinary.com/dymhvau8n/image/upl...,https://res.cloudinary.com/dymhvau8n/image/upl...,2024-06-15 13:03:33.662,2024-06-15 13:03:33.662,NaN
1,2,silver,50000,https://res.cloudinary.com/dymhvau8n/image/upl...,https://res.cloudinary.com/dymhvau8n/image/upl...,2024-06-15 13:03:33.666,2024-06-15 13:03:33.666,NaN
2,3,gold,150000,https://res.cloudinary.com/dymhvau8n/image/upl...,https://res.cloudinary.com/dymhvau8n/image/upl...,2024-06-15 13:03:33.672,2024-06-15 13:03:33.672,NaN
3,4,Platinum,300000,https://res.cloudinary.com/dymhvau8n/image/upl...,https://res.cloudinary.com/dymhvau8n/image/upl...,2024-06-15 13:03:33.677,2024-06-19 14:17:00.293,NaN


### Table admins

In [6]:
df_admins = pd.read_csv("data_database/admins.csv")
df_admins.head()

,id,name,email,password,role,image_url,created_at,updated_at,deleted_at
0,AD0001,John Doe Senior,john.doe.sr@gmail.com,$2a$12$1DTaG7rUzJCOgfar0HjWYOSLOuETS1kT0AN0KS0...,super admin,http://example.com/,2024-06-14 00:00:00.000,2024-06-15 13:03:33.534,NaN
1,AD0002,John Doe Sok Junior,john.doe.s.sr@gmail.com,$2a$12$1DTaG7rUzJCOgfar0HjWYOSLOuETS1kT0AN0KS0...,admin,http://example.com/,2024-06-14 00:00:00.000,2024-06-15 13:03:33.539,NaN
2,AD0003,Admin Ell,ell@gmail.com,$2a$12$ZAKLUJ8WT62wMB9pPqmMqeE3wzC5qRKta.pU1jJ...,admin,https://res.cloudinary.com/dlbbsdd3a/image/upl...,2024-06-16 12:12:22.910,2024-06-16 12:12:22.910,2024-06-18 13:50:17.095
3,AD0004,Test,test@gmail.com,$2a$12$Ye2.tsYyI5.Yo5MxXLVWeOpcMVbQgoVcnXlLaPp...,admin,https://res.cloudinary.com/dlbbsdd3a/image/upl...,2024-06-18 12:14:49.166,2024-06-18 12:14:49.166,2024-06-18 12:43:42.986
4,AD0005,Test lagi,lagi@gmail.com,$2a$12$Q5W0mTuTbuntPMVBzV2KxO5torhZpZF.riypUuG...,admin,https://res.cloudinary.com/dlbbsdd3a/image/upl...,2024-06-18 12:33:17.352,2024-06-18 12:33:17.352,2024-06-18 12:43:45.921


### Table article_categories

In [7]:
df_article_categories = pd.read_csv("data_database/article_categories.csv")
df_article_categories.head()

,id,article_id,waste_category_id,content_category_id,created_at,updated_at,deleted_at
0,1,ART0001,5,4,2024-06-11 15:07:04.000,2024-06-15 13:03:36.447,NaN
1,2,ART0001,11,2,2024-01-31 09:42:00.000,2024-06-15 13:03:36.452,NaN
2,3,ART0001,3,2,2024-01-17 01:43:07.000,2024-06-15 13:03:36.457,NaN
3,4,ART0002,11,4,2024-06-07 20:06:57.000,2024-06-15 13:03:36.461,NaN
4,5,ART0002,6,4,2024-05-17 14:38:11.000,2024-06-15 13:03:36.465,NaN


### Table article_comments

In [8]:
df_article_comments = pd.read_csv("data_database/article_comments.csv")
df_article_comments.head()

,id,user_id,article_id,comment,created_at,updated_at,deleted_at
0,1,USR0006,ART0001,Envious condemned Nepalese the utterly.,2024-05-06 04:22:34.671,2024-06-18 12:11:19.659,NaN
1,2,USR0013,ART0001,Would election way Dutch none.,2024-04-18 03:05:16.006,2024-06-18 12:11:19.662,NaN
2,3,USR0015,ART0001,Down little without myself time.,2024-03-16 12:50:49.073,2024-06-18 12:11:19.669,NaN
3,4,USR0042,ART0001,Myself troupe ouch work therefore.,2024-05-07 16:54:50.618,2024-06-18 12:11:19.673,NaN
4,5,USR0019,ART0001,Is this sunshine as any.,2024-03-09 23:30:22.960,2024-06-18 12:11:19.677,NaN


### Table article_sections

In [9]:
df_article_sections = pd.read_csv("data_database/article_sections.csv")
df_article_sections.head()

,id,article_id,title,description,image_url,created_at,updated_at,deleted_at
0,1,ART0001,First heavy sleep some she they.,Gifted man where. Today racism it.,https://picsum.photos/640/480,2024-01-28 14:42:26.000,2024-06-15 13:03:35.696,NaN
1,2,ART0001,She whose whose which am which.,Tighten we unlock. Everyone gang great.,https://picsum.photos/640/480,2024-04-21 08:14:39.000,2024-06-15 13:03:35.700,NaN
2,3,ART0002,Whose of into all scarcely her.,Themselves someone anyone. Whomever child heav...,https://picsum.photos/640/480,2024-01-19 04:11:55.000,2024-06-15 13:03:35.703,NaN
3,4,ART0002,Crawl his to besides next that.,That ring were. Up stay dangerous.,https://picsum.photos/640/480,2024-06-13 07:28:32.000,2024-06-15 13:03:35.708,NaN
4,5,ART0002,Within Welsh why what instance hug.,But left when. Sometimes honestly boy.,https://picsum.photos/640/480,2024-05-17 19:37:46.000,2024-06-15 13:03:35.713,NaN


### Table articles

In [10]:
df_articles = pd.read_csv("data_database/articles.csv")
df_articles.head()

,id,title,description,thumbnail_url,author_id,created_at,updated_at,deleted_at
0,ART0001,Equipment panda little her board disturbed.,Out guilt nightly. This tonight greedily.,https://picsum.photos/640/480,AD0001,2024-01-17 04:11:43.000,2024-06-15 13:03:35.485,NaN
1,ART0002,Luxury any edify kid why his.,What aha team. Such when neither.,https://picsum.photos/640/480,AD0001,2024-05-09 20:22:51.000,2024-06-15 13:03:35.493,NaN
2,ART0003,Rainbow marry his edify lastly soak.,At in a. Along those poverty.,https://picsum.photos/640/480,AD0001,2024-03-22 15:53:55.000,2024-06-15 13:03:35.497,NaN
3,ART0004,That from game daily fade herself.,Am calm abundant. She catalog those.,https://picsum.photos/640/480,AD0001,2024-02-25 11:41:00.000,2024-06-15 13:03:35.501,NaN
4,ART0005,Instance live positively exactly it it.,Never what rice. These fast mercy.,https://picsum.photos/640/480,AD0001,2024-04-06 03:12:06.000,2024-06-15 13:03:35.505,NaN


### Table comments

In [11]:
df_comments = pd.read_csv("data_database/comments.csv")
df_comments.head()

,id,video_id,user_id,comment,created_at,updated_at,deleted_at
0,1501,1,USR0043,Have they next those constantly.,2024-04-02 06:26:39.948,2024-06-18 12:11:26.250,NaN
1,1502,1,USR0018,Which trip myself that some.,2024-02-23 19:26:14.916,2024-06-18 12:11:26.258,NaN
2,1503,1,USR0023,Whose whomever these mine why.,2024-05-15 16:03:19.458,2024-06-18 12:11:26.263,NaN
3,1504,1,USR0039,What been yearly could number.,2024-03-05 11:19:29.583,2024-06-18 12:11:26.268,NaN
4,1505,1,USR0046,Other firstly of their horror.,2024-02-06 20:36:44.365,2024-06-18 12:11:26.272,NaN


### Table content_categories

In [12]:
df_content_categories = pd.read_csv("data_database/content_categories.csv")
df_content_categories.head()

,id,name,created_at,updated_at,deleted_at
0,1,tips,2024-06-15 13:03:33.640,2024-06-15 13:03:33.640,NaN
1,2,daur ulang,2024-06-15 13:03:33.644,2024-06-15 13:03:33.644,NaN
2,3,tutorial,2024-06-15 13:03:33.649,2024-06-15 13:03:33.649,NaN
3,4,edukasi,2024-06-15 13:03:33.652,2024-06-15 13:03:33.652,NaN
4,5,kampanye,2024-06-15 13:03:33.657,2024-06-15 13:03:33.657,NaN


### Table custom_data

In [13]:
df_custom_data = pd.read_csv("data_database/custom_data.csv")
df_custom_data.head()

,id,topic,description,created_at,updated_at,deleted_at
0,CDT0001,Daur Ulang Plastik,Proses daur ulang plastik melibatkan pengumpul...,2024-06-15 13:03:33.958,2024-06-15 13:03:33.958,NaN
1,CDT0002,Pemanfaatan Sampah Organik,Sampah organik seperti sisa makanan dan daun d...,2024-06-15 13:03:33.963,2024-06-15 13:03:33.963,NaN
2,CDT0003,Pengelolaan Sampah Elektronik,"Sampah elektronik seperti ponsel lama, kompute...",2024-06-15 13:03:33.968,2024-06-15 13:03:33.968,NaN
3,CDT0004,Kompetisi Pengurangan Sampah,Kompetisi ini mengajak masyarakat untuk mengur...,2024-06-15 13:03:33.971,2024-06-15 13:03:33.971,NaN
4,CDT0005,Melaporkan Sampah yang Tidak pada Tempatnya,Pengguna aplikasi dapat melaporkan sampah yang...,2024-06-15 13:03:33.976,2024-06-15 13:03:33.976,NaN


### Table faqs

In [14]:
df_faqs = pd.read_csv("data_database/faqs.csv")
df_faqs.head()

,id,category,question,answer,created_at,updated_at,deleted_at
0,FAQ01,profil,Bagaimana cara saya memperbarui informasi prof...,Anda dapat memperbarui informasi profil Anda m...,2024-06-15 13:03:33.867,2024-06-15 13:03:33.867,NaN
1,FAQ02,profil,Apakah saya bisa mengubah alamat email yang su...,"Ya, Anda bisa mengubah alamat email Anda melal...",2024-06-15 13:03:33.871,2024-06-15 13:03:33.871,NaN
2,FAQ03,profil,Bagaimana cara mengganti foto profil saya?,"Untuk mengganti foto profil, buka 'Profil Saya...",2024-06-15 13:03:33.875,2024-06-15 13:03:33.875,NaN
3,FAQ04,littering,Bagaimana cara melaporkan sampah yang tidak pa...,Anda dapat melaporkan sampah yang tidak pada t...,2024-06-15 13:03:33.879,2024-06-15 13:03:33.879,NaN
4,FAQ05,littering,Apakah ada sanksi bagi yang membuang sampah se...,"Ya, sesuai dengan peraturan daerah, membuang s...",2024-06-15 13:03:33.883,2024-06-15 13:03:33.883,NaN


### Table report_images

In [15]:
df_report_images = pd.read_csv("data_database/report_images.csv")
df_report_images.head()

,id,report_id,image_url,created_at,updated_at,deleted_at
0,0022888b-ad1d-4c46-a76d-db66e455ec98,RPT0030,https://picsum.photos/640/480,2024-06-15 13:03:38.395,2024-06-15 13:03:38.395,NaN
1,00f3273e-4ff0-473a-bf84-51e16c7c79ca,RPT0061,https://picsum.photos/640/480,2024-06-15 13:03:38.905,2024-06-15 13:03:38.905,NaN
2,02b5fae2-4365-45c1-a73c-4ae7ce1f7a09,RPT0004,https://picsum.photos/640/480,2024-06-15 13:03:37.972,2024-06-15 13:03:37.972,NaN
3,02fde631-b10d-4cae-a074-264669dd2beb,RPT0050,https://picsum.photos/640/480,2024-06-15 13:03:38.716,2024-06-15 13:03:38.716,NaN
4,0315b225-e268-44ac-9396-fc28559c465d,RPT0072,https://picsum.photos/640/480,2024-06-15 13:03:39.086,2024-06-15 13:03:39.086,NaN


### Table report_waste_materials

In [16]:
df_report_waste_materials = pd.read_csv("data_database/report_waste_materials.csv")
df_report_waste_materials.head()

,id,report_id,waste_material_id,created_at,updated_at,deleted_at
0,028b3c7e-fb45-417a-8f67-103d4e600a7f,RPT0036,MTR08,2024-06-15 13:03:37.575,2024-06-15 13:03:37.575,NaN
1,04ad45df-cda8-4319-a5dd-223872e68f07,RPT0022,MTR02,2024-06-15 13:03:37.503,2024-06-15 13:03:37.503,NaN
2,0594eac5-c6bb-43c9-86ee-7fd19c833b33,RPT0055,MTR04,2024-06-15 13:03:37.688,2024-06-15 13:03:37.688,NaN
3,06cc7d48-d03b-4d96-aa16-74496f30af5f,RPT0089,MTR01,2024-06-15 13:03:37.845,2024-06-15 13:03:37.845,NaN
4,07cca3b6-6d2c-431b-ae1b-32addae6ca36,RPT0059,MTR03,2024-06-15 13:03:37.730,2024-06-15 13:03:37.730,NaN


### Table reports

In [17]:
df_reports = pd.read_csv("data_database/reports.csv")
df_reports.head()

,id,author_id,report_type,title,description,waste_type,latitude,longitude,address,city,province,status,reason,created_at,updated_at,deleted_at
0,RPT0001,USR0007,rubbish,Tender now everything had quietly recently.,These monthly e.g.. Pack each ever.,sampah kering,88.244187,0.302420,Jalan Slamet Riyadi,Solo,Jawa Tengah,approve,NaN,2024-04-24 11:03:57.000,2024-06-19 16:51:44.096,NaN
1,RPT0002,USR0027,rubbish,Theirs murder respond you smile doubtfully.,Of already me. Block hurriedly man.,"sampah basah,sampah kering",61.387277,-168.170093,Jalan Jendral Sudirman,Jakarta,DKI Jakarta,reject,test,2024-01-17 11:27:05.000,2024-06-19 16:51:44.228,NaN
2,RPT0003,USR0036,littering,Depending your occasionally sit he party.,Moment occasionally honesty. Badly club chair.,organik,-9.808979,16.569601,Jalan Pahlawan,Denpasar,Bali,reject,Laporan Tidak Lengkap,2024-06-23 16:11:34.000,2024-06-19 16:36:23.749,NaN
3,RPT0004,USR0036,littering,Where there one mob aha out.,Amused unusual bale. However these that.,berbahaya,83.771301,-128.691805,Jalan Riau,Bandung,Jawa Barat,reject,Could her its conclude as.,2024-06-24 21:06:40.000,2024-06-15 13:03:36.990,NaN
4,RPT0005,USR0015,rubbish,Whom who return cast finally anything.,Omen these down. Consequently flick where.,sampah basah,8.424071,154.131695,Jalan Imam Bonjol,Semarang,Jawa Tengah,reject,Foto kurang jelas,2024-04-21 12:27:43.000,2024-06-16 06:47:44.836,NaN


### Table task_challenges

In [18]:
df_task_challenges = pd.read_csv("data_database/task_challenges.csv")
df_task_challenges.head()

,id,title,description,thumbnail,start_date,end_date,point,status,admin_id,created_at,updated_at,deleted_at
0,TM0001,Us philosophy crowd any utterly about.,Therefore have whom. Below furthermore every.,https://picsum.photos/640/480,2024-03-20 14:40:03,2024-03-27 14:40:03,2277,0,AD0001,2024-02-24 17:26:52.000,2024-06-16 00:00:00.008,NaN
1,TM0002,Him head sneeze one yourselves life.,Brilliance firstly occasionally. Who occasiona...,https://picsum.photos/640/480,2024-05-08 01:11:00,2024-05-11 01:11:00,1891,0,AD0001,2024-03-19 14:39:11.000,2024-06-16 00:00:00.008,NaN
2,TM0003,Accordingly along crew everything nightly due.,Will then a. In team everyone.,https://picsum.photos/640/480,2024-01-03 10:16:00,2024-01-08 10:16:00,966,0,AD0001,2024-01-01 03:15:46.000,2024-06-16 00:00:00.008,NaN
3,TM0004,Yourselves child mob break those energetic.,There moreover chase. Lots those bravo.,https://picsum.photos/640/480,2024-01-13 14:28:14,2024-01-16 14:28:14,955,0,AD0001,2024-01-09 22:50:31.000,2024-06-16 00:00:00.008,NaN
4,TM0005,Summation dangerous purely mine once ski.,So far just. May so these.,https://picsum.photos/640/480,2024-03-27 19:13:24,2024-04-02 19:13:24,1616,0,AD0001,2024-02-13 01:44:44.000,2024-06-16 00:00:00.008,NaN


### Table task_steps

In [19]:
df_task_steps = pd.read_csv("data_database/task_steps.csv")
df_task_steps.head()

,id,task_challenge_id,title,description,created_at,updated_at,deleted_at
0,1,TM0001,Step 1,These why you. Band sorrow theirs.,2024-01-23 07:51:11.000,2024-06-15 13:03:34.221,NaN
1,2,TM0001,Step 2,In off him. Besides that his.,2024-01-06 15:19:29.000,2024-06-15 13:03:34.225,NaN
2,3,TM0001,Step 3,Such in would. This should indeed.,2024-02-17 03:44:13.000,2024-06-15 13:03:34.230,NaN
3,4,TM0001,Step 4,Double their alternatively. Without ours below.,2024-02-20 05:12:57.000,2024-06-15 13:03:34.234,NaN
4,5,TM0002,Step 1,From summation congregation. Have e.g. even.,2024-04-12 04:22:38.000,2024-06-15 13:03:34.239,NaN


### Table user_task_challenges

In [20]:
df_user_task_challenges = pd.read_csv("data_database/user_task_challenges.csv")
df_user_task_challenges.head()

,id,user_id,task_challenge_id,status_progress,status_accept,description_image,point,reason,accepted_at,created_at,updated_at,deleted_at
0,UT0001,USR0001,TM0018,in_progress,need_rivew,NaN,0,NaN,2024-06-15 14:07:14,2024-06-15 14:07:13.863,2024-06-15 14:07:13.863,NaN
1,UT0002,USR0001,TM0010,in_progress,need_rivew,NaN,0,NaN,2024-06-15 16:53:09,2024-06-15 16:53:08.910,2024-06-15 16:53:08.910,NaN
2,UT0003,USR0001,TM0017,in_progress,need_rivew,NaN,0,NaN,2024-06-15 16:58:23,2024-06-15 16:58:23.310,2024-06-15 16:58:23.310,NaN
3,UT0004,USR0001,TM0015,in_progress,need_rivew,NaN,0,NaN,2024-06-16 01:45:44,2024-06-16 01:45:43.922,2024-06-16 01:45:43.922,NaN
4,UT0005,USR0034,TM0009,done,accept,Test,588,NaN,2024-06-18 15:31:19,2024-06-18 13:42:58.948,2024-06-18 15:31:19.353,NaN


### Table user_task_images

In [21]:
df_user_task_images = pd.read_csv("data_database/user_task_images.csv")
df_user_task_images.head()

,id,user_task_challenge_id,image_url,created_at,updated_at,deleted_at
0,1,UT0005,https://res.cloudinary.com/dlbbsdd3a/image/upl...,2024-06-18 14:22:29.201,2024-06-18 14:22:29.201,NaN
1,2,UT0005,https://res.cloudinary.com/dlbbsdd3a/image/upl...,2024-06-18 14:22:29.202,2024-06-18 14:22:29.202,NaN
2,3,UT0006,https://res.cloudinary.com/dlbbsdd3a/image/upl...,2024-06-18 14:52:31.495,2024-06-18 14:52:31.495,NaN
3,4,UT0006,https://res.cloudinary.com/dlbbsdd3a/image/upl...,2024-06-18 14:52:31.496,2024-06-18 14:52:31.496,NaN
4,5,UT0018,https://res.cloudinary.com/dlbbsdd3a/image/upl...,2024-06-19 09:15:50.396,2024-06-19 09:15:50.396,NaN


### Table user_task_steps

In [22]:
df_user_task_steps = pd.read_csv("data_database/user_task_steps.csv")
df_user_task_steps.head()

,id,user_task_challenge_id,task_step_id,completed,created_at,updated_at,deleted_at
0,1,UT0001,60,1,2024-06-15 14:07:13.864,2024-06-16 03:25:28.790,NaN
1,2,UT0001,61,1,2024-06-15 14:07:13.865,2024-06-16 06:18:49.441,NaN
2,3,UT0001,62,1,2024-06-15 14:07:13.867,2024-06-16 06:24:01.198,NaN
3,4,UT0002,32,1,2024-06-15 16:53:08.911,2024-06-16 06:37:55.764,NaN
4,5,UT0002,33,1,2024-06-15 16:53:08.912,2024-06-16 06:38:41.787,NaN


### Table users

In [23]:
df_users = pd.read_csv("data_database/users.csv")
df_users.head()

,id,name,email,password,point,gender,birth_date,address,picture_url,otp,is_verified,badge,created_at,updated_at,deleted_at
0,USR0001,Clinton Swaniawski,joshuahmante@kassulke.com,$2a$12$5KC.m.K50I2uW4xVUDxQyOrLUraC1kbr88y2/Le...,386284,perempuan,1998-06-20 02:24:22.828,"Jalan Malioboro, Yogyakarta, Indonesia",https://picsum.photos/200/200,570177,1,https://res.cloudinary.com/dymhvau8n/image/upl...,2024-03-24 06:25:37.000,2024-06-15 13:03:32.991,2024-06-16 17:35:04.506
1,USR0002,Joelle Mann,ollieflatley@kunze.biz,$2a$12$5KC.m.K50I2uW4xVUDxQyOrLUraC1kbr88y2/Le...,320221,perempuan,2003-05-16 21:48:45.332,"Jalan Tunjungan, Surabaya, Indonesia",https://picsum.photos/200/200,137299,1,https://res.cloudinary.com/dymhvau8n/image/upl...,2024-06-01 18:01:58.000,2024-06-15 13:03:32.995,NaN
2,USR0003,Austen Pfeffer,brodyhoeger@halvorson.biz,$2a$12$5KC.m.K50I2uW4xVUDxQyOrLUraC1kbr88y2/Le...,47791,laki-laki,1976-04-17 19:47:54.245,"Jalan Gatot Subroto, Jakarta, Indonesia",https://res.cloudinary.com/dlbbsdd3a/image/upl...,556740,1,https://res.cloudinary.com/dymhvau8n/image/upl...,2024-06-17 04:21:57.000,2024-06-19 09:59:08.101,NaN
3,USR0004,Joanie Turcotte,metapredovic@daniel.org,$2a$12$5KC.m.K50I2uW4xVUDxQyOrLUraC1kbr88y2/Le...,164319,perempuan,1994-12-02 00:00:00.000,"Jalan Panglima Polim, Jakarta, Indonesia",https://res.cloudinary.com/dlbbsdd3a/image/upl...,419910,1,https://res.cloudinary.com/dymhvau8n/image/upl...,2024-06-02 15:56:00.000,2024-06-19 14:40:22.949,NaN
4,USR0005,Toney Kirlin,lorenhermann@gorczany.info,$2a$12$5KC.m.K50I2uW4xVUDxQyOrLUraC1kbr88y2/Le...,46040,laki-laki,1976-05-27 22:01:42.085,"Jalan Panglima Polim, Jakarta, Indonesia",https://picsum.photos/200/200,440526,1,https://res.cloudinary.com/dymhvau8n/image/upl...,2024-02-05 07:36:27.000,2024-06-15 13:03:33.010,NaN


### Table video_categories

In [24]:
df_video_categories = pd.read_csv("data_database/video_categories.csv")
df_video_categories.head()

,id,video_id,content_category_id,waste_category_id,created_at,updated_at,deleted_at
0,1,1,2,2,2024-03-05 03:07:20.000,2024-06-15 13:03:34.880,NaN
1,2,1,1,12,2024-01-06 13:00:12.000,2024-06-15 13:03:34.885,NaN
2,3,2,2,10,2024-03-09 23:02:34.000,2024-06-15 13:03:34.895,NaN
3,4,2,2,1,2024-04-14 09:32:03.000,2024-06-15 13:03:34.899,NaN
4,5,2,2,2,2024-05-01 09:11:40.000,2024-06-15 13:03:34.904,NaN


### Table videos

In [25]:
df_videos = pd.read_csv("data_database/videos.csv")
df_videos.head()

,id,title,description,thumbnail,link,viewer,created_at,updated_at,deleted_at
0,1,lah update,Video Description,https://picsum.photos/640/480,https://www.youtube.com/watch?v=NH9yuZUrJVc,658331,2024-04-07 14:49:15.000,2024-06-16 18:00:49.986,2024-06-16 18:12:34.522
1,2,She within way trust quarterly others.,Upstairs inside for. Already party here.,https://picsum.photos/640/480,https://www.youtube.com/watch?v=CGd3lgxReFE,0,2024-01-02 06:14:54.000,2024-06-15 13:03:34.664,2024-06-16 18:10:26.261
2,3,Of lovely huh off their crime.,Amused under what. Itself herself company.,https://picsum.photos/640/480,https://www.youtube.com/watch?v=CGd3lgxReFE,199917,2024-04-20 07:15:07.000,2024-06-19 09:49:13.031,NaN
3,4,Shall towards anyone begin did quiver.,Page no little. Nightly had finally.,https://picsum.photos/640/480,https://www.youtube.com/watch?v=CGd3lgxReFE,199917,2024-04-23 04:02:40.000,2024-06-19 09:49:12.818,NaN
4,5,Thing are lie cabinet select whoa.,In behind example. Party everything including.,https://picsum.photos/640/480,https://www.youtube.com/watch?v=CGd3lgxReFE,199917,2024-01-03 11:21:37.000,2024-06-19 09:49:20.075,NaN


### Table waste_categories

In [26]:
df_waste_categories = pd.read_csv("data_database/waste_categories.csv")
df_waste_categories.head()

,id,name,created_at,updated_at,deleted_at
0,1,plastik,2024-06-15 13:03:33.587,2024-06-15 13:03:33.587,NaN
1,2,besi,2024-06-15 13:03:33.591,2024-06-15 13:03:33.591,NaN
2,3,kaca,2024-06-15 13:03:33.595,2024-06-15 13:03:33.595,NaN
3,4,organik,2024-06-15 13:03:33.599,2024-06-15 13:03:33.599,NaN
4,5,kayu,2024-06-15 13:03:33.603,2024-06-15 13:03:33.603,NaN


### Table waste_materials

In [27]:
df_waste_materials = pd.read_csv("data_database/waste_materials.csv")
df_waste_materials.head()

,id,type,created_at,updated_at,deleted_at
0,MTR01,plastik,2024-06-15 13:03:33.543,2024-06-15 13:03:33.543,NaN
1,MTR02,kaca,2024-06-15 13:03:33.548,2024-06-15 13:03:33.548,NaN
2,MTR03,kayu,2024-06-15 13:03:33.553,2024-06-15 13:03:33.553,NaN
3,MTR04,kertas,2024-06-15 13:03:33.558,2024-06-15 13:03:33.558,NaN
4,MTR05,baterai,2024-06-15 13:03:33.562,2024-06-15 13:03:33.562,NaN


# TRANSFORM

## Mengecek tipe data

In [28]:
data_files = [
    ("about_us_images.csv", df_about_us_images),
    ("about_us.csv", df_about_us),
    ("achievements.csv", df_achievements),
    ("admins.csv", df_admins),
    ("article_categories.csv", df_article_categories),
    ("article_comments.csv", df_article_comments),
    ("article_sections.csv", df_article_sections),
    ("articles.csv", df_articles),
    ("comments.csv", df_comments),
    ("content_categories.csv", df_content_categories),
    ("custom_data.csv", df_custom_data),
    ("faqs.csv", df_faqs),
    ("report_images.csv", df_report_images),
    ("report_waste_materials.csv", df_report_waste_materials),
    ("reports.csv", df_reports),
    ("task_challenges.csv", df_task_challenges),
    ("task_steps.csv", df_task_steps),
    ("user_task_challenges.csv", df_user_task_challenges),
    ("user_task_images.csv", df_user_task_images),
    ("user_task_steps.csv", df_user_task_steps),
    ("users.csv", df_users),
    ("video_categories.csv", df_video_categories),
    ("videos.csv", df_videos),
    ("waste_categories.csv", df_waste_categories),
    ("waste_materials.csv", df_waste_materials),
]

for file_name, data in data_files:
    print(f"\n{file_name}:")
    data.info()


about_us_images.csv:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           26 non-null     object 
 1   about_us_id  26 non-null     object 
 2   name         22 non-null     object 
 3   image_url    26 non-null     object 
 4   created_at   26 non-null     object 
 5   updated_at   26 non-null     object 
 6   deleted_at   0 non-null      float64
dtypes: float64(1), object(6)
memory usage: 1.6+ KB

about_us.csv:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           16 non-null     object 
 1   category     16 non-null     object 
 2   title        16 non-null     object 
 3   description  16 non-null     object 
 4   created_at   16 non-null     object 
 5   updated_at   16 non-null     o

## Menyesuaikan tipe data

### Table about_us_images

In [29]:
df_about_us_images['created_at'] = pd.to_datetime(df_about_us_images['created_at'], errors='coerce')
df_about_us_images['updated_at'] = pd.to_datetime(df_about_us_images['updated_at'], errors='coerce')
df_about_us_images['deleted_at'] = pd.to_datetime(df_about_us_images['deleted_at'], errors='coerce')

df_about_us_images.to_csv('staging_area/about_us_images.csv', index=False)

In [30]:
df_about_us_images.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           26 non-null     object        
 1   about_us_id  26 non-null     object        
 2   name         22 non-null     object        
 3   image_url    26 non-null     object        
 4   created_at   26 non-null     datetime64[ns]
 5   updated_at   26 non-null     datetime64[ns]
 6   deleted_at   0 non-null      datetime64[ns]
dtypes: datetime64[ns](3), object(4)
memory usage: 1.6+ KB


### Table about_us

In [31]:
df_about_us['created_at'] = pd.to_datetime(df_about_us['created_at'], errors='coerce')
df_about_us['updated_at'] = pd.to_datetime(df_about_us['updated_at'], errors='coerce')
df_about_us['deleted_at'] = pd.to_datetime(df_about_us['deleted_at'], errors='coerce')

df_about_us.to_csv('staging_area/about_us.csv', index=False)

In [32]:
df_about_us.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           16 non-null     object        
 1   category     16 non-null     object        
 2   title        16 non-null     object        
 3   description  16 non-null     object        
 4   created_at   16 non-null     datetime64[ns]
 5   updated_at   16 non-null     datetime64[ns]
 6   deleted_at   0 non-null      datetime64[ns]
dtypes: datetime64[ns](3), object(4)
memory usage: 1.0+ KB


### Table achievements

In [33]:
df_achievements['created_at'] = pd.to_datetime(df_achievements['created_at'], errors='coerce')
df_achievements['updated_at'] = pd.to_datetime(df_achievements['updated_at'], errors='coerce')
df_achievements['deleted_at'] = pd.to_datetime(df_achievements['deleted_at'], errors='coerce')

df_achievements.to_csv('staging_area/achievements.csv', index=False)

In [34]:
df_achievements.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              4 non-null      int64         
 1   level           4 non-null      object        
 2   target_point    4 non-null      int64         
 3   badge_url       4 non-null      object        
 4   badge_url_user  4 non-null      object        
 5   created_at      4 non-null      datetime64[ns]
 6   updated_at      4 non-null      datetime64[ns]
 7   deleted_at      0 non-null      datetime64[ns]
dtypes: datetime64[ns](3), int64(2), object(3)
memory usage: 388.0+ bytes


### Table admins

In [35]:
df_admins['created_at'] = pd.to_datetime(df_admins['created_at'], errors='coerce')
df_admins['updated_at'] = pd.to_datetime(df_admins['updated_at'], errors='coerce')
df_admins['deleted_at'] = pd.to_datetime(df_admins['deleted_at'], errors='coerce')

df_admins.to_csv('staging_area/admins.csv', index=False)

In [36]:
df_admins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id          20 non-null     object        
 1   name        20 non-null     object        
 2   email       20 non-null     object        
 3   password    20 non-null     object        
 4   role        20 non-null     object        
 5   image_url   20 non-null     object        
 6   created_at  20 non-null     datetime64[ns]
 7   updated_at  20 non-null     datetime64[ns]
 8   deleted_at  15 non-null     datetime64[ns]
dtypes: datetime64[ns](3), object(6)
memory usage: 1.5+ KB


### Table article_categories

In [37]:
df_article_categories['created_at'] = pd.to_datetime(df_article_categories['created_at'], errors='coerce')
df_article_categories['updated_at'] = pd.to_datetime(df_article_categories['updated_at'], errors='coerce')
df_article_categories['deleted_at'] = pd.to_datetime(df_article_categories['deleted_at'], errors='coerce')

df_article_categories.to_csv('staging_area/article_categories.csv', index=False)

In [38]:
df_article_categories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127 entries, 0 to 126
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id                   127 non-null    int64         
 1   article_id           127 non-null    object        
 2   waste_category_id    127 non-null    int64         
 3   content_category_id  127 non-null    int64         
 4   created_at           127 non-null    datetime64[ns]
 5   updated_at           127 non-null    datetime64[ns]
 6   deleted_at           23 non-null     datetime64[ns]
dtypes: datetime64[ns](3), int64(3), object(1)
memory usage: 7.1+ KB


### Table article_comments

In [39]:
df_article_comments['created_at'] = pd.to_datetime(df_article_comments['created_at'], errors='coerce')
df_article_comments['updated_at'] = pd.to_datetime(df_article_comments['updated_at'], errors='coerce')
df_article_comments['deleted_at'] = pd.to_datetime(df_article_comments['deleted_at'], errors='coerce')

df_article_comments.to_csv('staging_area/article_comments.csv', index=False)

In [40]:
df_article_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1505 entries, 0 to 1504
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id          1505 non-null   int64         
 1   user_id     1505 non-null   object        
 2   article_id  1505 non-null   object        
 3   comment     1503 non-null   object        
 4   created_at  1505 non-null   datetime64[ns]
 5   updated_at  1505 non-null   datetime64[ns]
 6   deleted_at  0 non-null      datetime64[ns]
dtypes: datetime64[ns](3), int64(1), object(3)
memory usage: 82.4+ KB


### Table article_sections

In [41]:
df_article_sections['created_at'] = pd.to_datetime(df_article_sections['created_at'], errors='coerce')
df_article_sections['updated_at'] = pd.to_datetime(df_article_sections['updated_at'], errors='coerce')
df_article_sections['deleted_at'] = pd.to_datetime(df_article_sections['deleted_at'], errors='coerce')

df_article_sections.to_csv('staging_area/article_sections.csv', index=False)

In [42]:
df_article_sections.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           200 non-null    int64         
 1   article_id   200 non-null    object        
 2   title        198 non-null    object        
 3   description  198 non-null    object        
 4   image_url    194 non-null    object        
 5   created_at   200 non-null    datetime64[ns]
 6   updated_at   200 non-null    datetime64[ns]
 7   deleted_at   26 non-null     datetime64[ns]
dtypes: datetime64[ns](3), int64(1), object(4)
memory usage: 12.6+ KB


### Table articles

In [43]:
df_articles['created_at'] = pd.to_datetime(df_articles['created_at'], errors='coerce')
df_articles['updated_at'] = pd.to_datetime(df_articles['updated_at'], errors='coerce')
df_articles['deleted_at'] = pd.to_datetime(df_articles['deleted_at'], errors='coerce')

df_articles.to_csv('staging_area/articles.csv', index=False)

In [44]:
df_articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             78 non-null     object        
 1   title          63 non-null     object        
 2   description    63 non-null     object        
 3   thumbnail_url  60 non-null     object        
 4   author_id      78 non-null     object        
 5   created_at     78 non-null     datetime64[ns]
 6   updated_at     78 non-null     datetime64[ns]
 7   deleted_at     27 non-null     datetime64[ns]
dtypes: datetime64[ns](3), object(5)
memory usage: 5.0+ KB


### Table comments

In [45]:
df_comments['created_at'] = pd.to_datetime(df_comments['created_at'], errors='coerce')
df_comments['updated_at'] = pd.to_datetime(df_comments['updated_at'], errors='coerce')
df_comments['deleted_at'] = pd.to_datetime(df_comments['deleted_at'], errors='coerce')

df_comments.to_csv('staging_area/comments.csv', index=False)

In [46]:
df_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1502 entries, 0 to 1501
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id          1502 non-null   int64         
 1   video_id    1502 non-null   int64         
 2   user_id     1502 non-null   object        
 3   comment     1502 non-null   object        
 4   created_at  1502 non-null   datetime64[ns]
 5   updated_at  1502 non-null   datetime64[ns]
 6   deleted_at  0 non-null      datetime64[ns]
dtypes: datetime64[ns](3), int64(2), object(2)
memory usage: 82.3+ KB


### Table content_categories

In [47]:
df_content_categories['created_at'] = pd.to_datetime(df_content_categories['created_at'], errors='coerce')
df_content_categories['updated_at'] = pd.to_datetime(df_content_categories['updated_at'], errors='coerce')
df_content_categories['deleted_at'] = pd.to_datetime(df_content_categories['deleted_at'], errors='coerce')

df_content_categories.to_csv('staging_area/content_categories.csv', index=False)

In [48]:
df_content_categories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id          5 non-null      int64         
 1   name        5 non-null      object        
 2   created_at  5 non-null      datetime64[ns]
 3   updated_at  5 non-null      datetime64[ns]
 4   deleted_at  0 non-null      datetime64[ns]
dtypes: datetime64[ns](3), int64(1), object(1)
memory usage: 332.0+ bytes


### Table custom_data

In [49]:
df_custom_data['created_at'] = pd.to_datetime(df_custom_data['created_at'], errors='coerce')
df_custom_data['updated_at'] = pd.to_datetime(df_custom_data['updated_at'], errors='coerce')
df_custom_data['deleted_at'] = pd.to_datetime(df_custom_data['deleted_at'], errors='coerce')

df_custom_data.to_csv('staging_area/custom_data.csv', index=False)

In [50]:
df_custom_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           28 non-null     object        
 1   topic        28 non-null     object        
 2   description  28 non-null     object        
 3   created_at   28 non-null     datetime64[ns]
 4   updated_at   28 non-null     datetime64[ns]
 5   deleted_at   15 non-null     datetime64[ns]
dtypes: datetime64[ns](3), object(3)
memory usage: 1.4+ KB


### Table faqs

In [51]:
df_faqs['created_at'] = pd.to_datetime(df_faqs['created_at'], errors='coerce')
df_faqs['updated_at'] = pd.to_datetime(df_faqs['updated_at'], errors='coerce')
df_faqs['deleted_at'] = pd.to_datetime(df_faqs['deleted_at'], errors='coerce')

df_faqs.to_csv('staging_area/faqs.csv', index=False)

In [52]:
df_faqs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id          21 non-null     object        
 1   category    21 non-null     object        
 2   question    21 non-null     object        
 3   answer      21 non-null     object        
 4   created_at  21 non-null     datetime64[ns]
 5   updated_at  21 non-null     datetime64[ns]
 6   deleted_at  0 non-null      datetime64[ns]
dtypes: datetime64[ns](3), object(4)
memory usage: 1.3+ KB


### Table report_images

In [53]:
df_report_images['created_at'] = pd.to_datetime(df_report_images['created_at'], errors='coerce')
df_report_images['updated_at'] = pd.to_datetime(df_report_images['updated_at'], errors='coerce')
df_report_images['deleted_at'] = pd.to_datetime(df_report_images['deleted_at'], errors='coerce')

df_report_images.to_csv('staging_area/report_images.csv', index=False)

In [54]:
df_report_images.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420 entries, 0 to 419
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id          420 non-null    object        
 1   report_id   420 non-null    object        
 2   image_url   420 non-null    object        
 3   created_at  420 non-null    datetime64[ns]
 4   updated_at  420 non-null    datetime64[ns]
 5   deleted_at  0 non-null      datetime64[ns]
dtypes: datetime64[ns](3), object(3)
memory usage: 19.8+ KB


### Table report_waste_materials

In [55]:
# Mapping nilai waste_material_id ke nama-nama material
mapping = {
    'MTR01': 'plastik',
    'MTR02': 'kaca',
    'MTR03': 'kayu',
    'MTR04': 'kertas',
    'MTR05': 'baterai',
    'MTR06': 'besi',
    'MTR07': 'limbah berbahaya',
    'MTR08': 'limbah beracun',
    'MTR09': 'sisa makanan',
    'MTR10': 'tak terdeteksi'
}

# Mengubah nilai kolom waste_material_id berdasarkan mapping
df_report_waste_materials['waste_material_id'] = df_report_waste_materials['waste_material_id'].map(mapping)

df_report_waste_materials.rename(columns={'waste_material_id': 'waste_material'}, inplace=True)

df_report_waste_materials['created_at'] = pd.to_datetime(df_report_waste_materials['created_at'], errors='coerce')
df_report_waste_materials['updated_at'] = pd.to_datetime(df_report_waste_materials['updated_at'], errors='coerce')
df_report_waste_materials['deleted_at'] = pd.to_datetime(df_report_waste_materials['deleted_at'], errors='coerce')

df_report_waste_materials.to_csv('staging_area/report_waste_materials.csv', index=False)

In [56]:
df_report_waste_materials.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162 entries, 0 to 161
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              162 non-null    object        
 1   report_id       162 non-null    object        
 2   waste_material  162 non-null    object        
 3   created_at      162 non-null    datetime64[ns]
 4   updated_at      162 non-null    datetime64[ns]
 5   deleted_at      0 non-null      datetime64[ns]
dtypes: datetime64[ns](3), object(3)
memory usage: 7.7+ KB


### Table reports

In [57]:
df_reports['created_at'] = pd.to_datetime(df_reports['created_at'], errors='coerce')
df_reports['updated_at'] = pd.to_datetime(df_reports['updated_at'], errors='coerce')
df_reports['deleted_at'] = pd.to_datetime(df_reports['deleted_at'], errors='coerce')

df_reports['waste_type'] = df_reports['waste_type'].str.replace(',', ', ')

df_reports.to_csv('staging_area/reports.csv', index=False)

In [58]:
df_reports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133 entries, 0 to 132
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           133 non-null    object        
 1   author_id    133 non-null    object        
 2   report_type  133 non-null    object        
 3   title        110 non-null    object        
 4   description  133 non-null    object        
 5   waste_type   133 non-null    object        
 6   latitude     133 non-null    float64       
 7   longitude    133 non-null    float64       
 8   address      133 non-null    object        
 9   city         133 non-null    object        
 10  province     133 non-null    object        
 11  status       133 non-null    object        
 12  reason       35 non-null     object        
 13  created_at   133 non-null    datetime64[ns]
 14  updated_at   133 non-null    datetime64[ns]
 15  deleted_at   0 non-null      datetime64[ns]
dtypes: datet

### Table task_challenges

In [59]:
df_task_challenges['created_at'] = pd.to_datetime(df_task_challenges['created_at'], errors='coerce')
df_task_challenges['updated_at'] = pd.to_datetime(df_task_challenges['updated_at'], errors='coerce')
df_task_challenges['deleted_at'] = pd.to_datetime(df_task_challenges['deleted_at'], errors='coerce')
df_task_challenges['start_date'] = pd.to_datetime(df_task_challenges['start_date'], errors='coerce')
df_task_challenges['end_date'] = pd.to_datetime(df_task_challenges['end_date'], errors='coerce')

df_task_challenges.to_csv('staging_area/task_challenges.csv', index=False)

In [60]:
df_task_challenges.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           23 non-null     object        
 1   title        23 non-null     object        
 2   description  23 non-null     object        
 3   thumbnail    23 non-null     object        
 4   start_date   23 non-null     datetime64[ns]
 5   end_date     23 non-null     datetime64[ns]
 6   point        23 non-null     int64         
 7   status       23 non-null     int64         
 8   admin_id     23 non-null     object        
 9   created_at   23 non-null     datetime64[ns]
 10  updated_at   23 non-null     datetime64[ns]
 11  deleted_at   11 non-null     datetime64[ns]
dtypes: datetime64[ns](5), int64(2), object(5)
memory usage: 2.3+ KB


### Table task_steps

In [61]:
df_task_steps['created_at'] = pd.to_datetime(df_task_steps['created_at'], errors='coerce')
df_task_steps['updated_at'] = pd.to_datetime(df_task_steps['updated_at'], errors='coerce')
df_task_steps['deleted_at'] = pd.to_datetime(df_task_steps['deleted_at'], errors='coerce')

df_task_steps.to_csv('staging_area/task_steps.csv', index=False)

In [62]:
df_task_steps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139 entries, 0 to 138
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 139 non-null    int64         
 1   task_challenge_id  139 non-null    object        
 2   title              139 non-null    object        
 3   description        139 non-null    object        
 4   created_at         139 non-null    datetime64[ns]
 5   updated_at         139 non-null    datetime64[ns]
 6   deleted_at         63 non-null     datetime64[ns]
dtypes: datetime64[ns](3), int64(1), object(3)
memory usage: 7.7+ KB


### Table user_task_challenges

In [63]:
df_user_task_challenges['accepted_at'] = pd.to_datetime(df_user_task_challenges['accepted_at'], errors='coerce')
df_user_task_challenges['created_at'] = pd.to_datetime(df_user_task_challenges['created_at'], errors='coerce')
df_user_task_challenges['updated_at'] = pd.to_datetime(df_user_task_challenges['updated_at'], errors='coerce')
df_user_task_challenges['deleted_at'] = pd.to_datetime(df_user_task_challenges['deleted_at'], errors='coerce')

df_user_task_challenges.to_csv('staging_area/user_task_challenges.csv', index=False)

In [64]:
df_user_task_challenges.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 31 non-null     object        
 1   user_id            31 non-null     object        
 2   task_challenge_id  31 non-null     object        
 3   status_progress    31 non-null     object        
 4   status_accept      31 non-null     object        
 5   description_image  11 non-null     object        
 6   point              31 non-null     int64         
 7   reason             2 non-null      object        
 8   accepted_at        31 non-null     datetime64[ns]
 9   created_at         31 non-null     datetime64[ns]
 10  updated_at         31 non-null     datetime64[ns]
 11  deleted_at         0 non-null      datetime64[ns]
dtypes: datetime64[ns](4), int64(1), object(7)
memory usage: 3.0+ KB


### Table user_task_images

In [65]:
df_user_task_images['created_at'] = pd.to_datetime(df_user_task_images['created_at'], errors='coerce')
df_user_task_images['updated_at'] = pd.to_datetime(df_user_task_images['updated_at'], errors='coerce')
df_user_task_images['deleted_at'] = pd.to_datetime(df_user_task_images['deleted_at'], errors='coerce')

df_user_task_images.to_csv('staging_area/user_task_images.csv', index=False)

In [66]:
df_user_task_images.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   id                      18 non-null     int64         
 1   user_task_challenge_id  18 non-null     object        
 2   image_url               18 non-null     object        
 3   created_at              18 non-null     datetime64[ns]
 4   updated_at              18 non-null     datetime64[ns]
 5   deleted_at              0 non-null      datetime64[ns]
dtypes: datetime64[ns](3), int64(1), object(2)
memory usage: 996.0+ bytes


### Table user_task_steps

In [67]:
df_user_task_steps['created_at'] = pd.to_datetime(df_user_task_steps['created_at'], errors='coerce')
df_user_task_steps['updated_at'] = pd.to_datetime(df_user_task_steps['updated_at'], errors='coerce')
df_user_task_steps['deleted_at'] = pd.to_datetime(df_user_task_steps['deleted_at'], errors='coerce')

df_user_task_steps.to_csv('staging_area/user_task_steps.csv', index=False)

In [68]:
df_user_task_steps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   id                      96 non-null     int64         
 1   user_task_challenge_id  96 non-null     object        
 2   task_step_id            96 non-null     int64         
 3   completed               96 non-null     int64         
 4   created_at              96 non-null     datetime64[ns]
 5   updated_at              96 non-null     datetime64[ns]
 6   deleted_at              0 non-null      datetime64[ns]
dtypes: datetime64[ns](3), int64(3), object(1)
memory usage: 5.4+ KB


### Table users

In [69]:
df_users['birth_date'] = pd.to_datetime(df_users['birth_date'], errors='coerce')
df_users['created_at'] = pd.to_datetime(df_users['created_at'], errors='coerce')
df_users['updated_at'] = pd.to_datetime(df_users['updated_at'], errors='coerce')
df_users['deleted_at'] = pd.to_datetime(df_users['deleted_at'], errors='coerce')

# Mengubah format tanggal hanya menampilkan tahun, bulan, dan tanggal pada kolom birth_date
df_users['birth_date'] = df_users['birth_date'].dt.strftime('%Y-%m-%d')

# Mapping URL badge ke nilai badge yang diinginkan
badge_mapping = {
    'https://res.cloudinary.com/dymhvau8n/image/upload/v1718189121/user_badge/htaemsjtlhfof7ww01ss.png': 'classic',
    'https://res.cloudinary.com/dymhvau8n/image/upload/v1718189221/user_badge/oespnjdgoynkairlutbk.png': 'silver',
    'https://res.cloudinary.com/dymhvau8n/image/upload/v1718189184/user_badge/jshs1s2fwevahgtvjkgj.png': 'gold',
    'https://res.cloudinary.com/dymhvau8n/image/upload/v1718188250/user_badge/icureiapdvtzyu5b99zu.png': 'platinum'
}

# Iterasi dan ganti nilai badge berdasarkan mapping
for index, row in df_users.iterrows():
    badge_url = row['badge']
    if badge_url in badge_mapping:
        df_users.at[index, 'badge'] = badge_mapping[badge_url]

df_users.to_csv('staging_area/users.csv', index=False)

In [70]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           58 non-null     object        
 1   name         58 non-null     object        
 2   email        58 non-null     object        
 3   password     58 non-null     object        
 4   point        58 non-null     int64         
 5   gender       58 non-null     object        
 6   birth_date   50 non-null     object        
 7   address      50 non-null     object        
 8   picture_url  52 non-null     object        
 9   otp          58 non-null     int64         
 10  is_verified  58 non-null     int64         
 11  badge        58 non-null     object        
 12  created_at   58 non-null     datetime64[ns]
 13  updated_at   58 non-null     datetime64[ns]
 14  deleted_at   3 non-null      datetime64[ns]
dtypes: datetime64[ns](3), int64(3), object(9)
memory usage: 6.9

### Table video_categories

In [71]:
df_video_categories['created_at'] = pd.to_datetime(df_video_categories['created_at'], errors='coerce')
df_video_categories['updated_at'] = pd.to_datetime(df_video_categories['updated_at'], errors='coerce')
df_video_categories['deleted_at'] = pd.to_datetime(df_video_categories['deleted_at'], errors='coerce')

df_video_categories.to_csv('staging_area/video_categories.csv', index=False)

In [72]:
df_video_categories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 135 entries, 0 to 134
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id                   135 non-null    int64         
 1   video_id             135 non-null    int64         
 2   content_category_id  135 non-null    int64         
 3   waste_category_id    135 non-null    int64         
 4   created_at           135 non-null    datetime64[ns]
 5   updated_at           135 non-null    datetime64[ns]
 6   deleted_at           17 non-null     datetime64[ns]
dtypes: datetime64[ns](3), int64(4)
memory usage: 7.5 KB


### Table videos

In [73]:
df_videos['created_at'] = pd.to_datetime(df_videos['created_at'], errors='coerce')
df_videos['updated_at'] = pd.to_datetime(df_videos['updated_at'], errors='coerce')
df_videos['deleted_at'] = pd.to_datetime(df_videos['deleted_at'], errors='coerce')

df_videos.to_csv('staging_area/videos.csv', index=False)

In [74]:
df_videos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           58 non-null     int64         
 1   title        58 non-null     object        
 2   description  58 non-null     object        
 3   thumbnail    58 non-null     object        
 4   link         58 non-null     object        
 5   viewer       58 non-null     int64         
 6   created_at   58 non-null     datetime64[ns]
 7   updated_at   58 non-null     datetime64[ns]
 8   deleted_at   6 non-null      datetime64[ns]
dtypes: datetime64[ns](3), int64(2), object(4)
memory usage: 4.2+ KB


### Table waste_categories

In [75]:
df_waste_categories['created_at'] = pd.to_datetime(df_waste_categories['created_at'], errors='coerce')
df_waste_categories['updated_at'] = pd.to_datetime(df_waste_categories['updated_at'], errors='coerce')
df_waste_categories['deleted_at'] = pd.to_datetime(df_waste_categories['deleted_at'], errors='coerce')

df_waste_categories.to_csv('staging_area/waste_categories.csv', index=False)

In [76]:
df_waste_categories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id          13 non-null     int64         
 1   name        13 non-null     object        
 2   created_at  13 non-null     datetime64[ns]
 3   updated_at  13 non-null     datetime64[ns]
 4   deleted_at  0 non-null      datetime64[ns]
dtypes: datetime64[ns](3), int64(1), object(1)
memory usage: 652.0+ bytes


### Table waste_materials

In [77]:
df_waste_materials['created_at'] = pd.to_datetime(df_waste_materials['created_at'], errors='coerce')
df_waste_materials['updated_at'] = pd.to_datetime(df_waste_materials['updated_at'], errors='coerce')
df_waste_materials['deleted_at'] = pd.to_datetime(df_waste_materials['deleted_at'], errors='coerce')

df_waste_materials.to_csv('staging_area/waste_materials.csv', index=False)

In [78]:
df_waste_materials.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id          10 non-null     object        
 1   type        10 non-null     object        
 2   created_at  10 non-null     datetime64[ns]
 3   updated_at  10 non-null     datetime64[ns]
 4   deleted_at  0 non-null      datetime64[ns]
dtypes: datetime64[ns](3), object(2)
memory usage: 532.0+ bytes


## Mengecek missing values

In [79]:
data_files = [
    ("about_us_images.csv", df_about_us_images),
    ("about_us.csv", df_about_us),
    ("achievements.csv", df_achievements),
    ("admins.csv", df_admins),
    ("article_categories.csv", df_article_categories),
    ("article_comments.csv", df_article_comments),
    ("article_sections.csv", df_article_sections),
    ("articles.csv", df_articles),
    ("comments.csv", df_comments),
    ("content_categories.csv", df_content_categories),
    ("custom_data.csv", df_custom_data),
    ("faqs.csv", df_faqs),
    ("report_images.csv", df_report_images),
    ("report_waste_materials.csv", df_report_waste_materials),
    ("reports.csv", df_reports),
    ("task_challenges.csv", df_task_challenges),
    ("task_steps.csv", df_task_steps),
    ("user_task_challenges.csv", df_user_task_challenges),
    ("user_task_images.csv", df_user_task_images),
    ("user_task_steps.csv", df_user_task_steps),
    ("users.csv", df_users),
    ("video_categories.csv", df_video_categories),
    ("videos.csv", df_videos),
    ("waste_categories.csv", df_waste_categories),
    ("waste_materials.csv", df_waste_materials),
]

for file_name, data in data_files:
    print(f"\n{file_name}:")
    missing_values = data.isnull().sum()
    print(missing_values)


about_us_images.csv:
id              0
about_us_id     0
name            4
image_url       0
created_at      0
updated_at      0
deleted_at     26
dtype: int64

about_us.csv:
id              0
category        0
title           0
description     0
created_at      0
updated_at      0
deleted_at     16
dtype: int64

achievements.csv:
id                0
level             0
target_point      0
badge_url         0
badge_url_user    0
created_at        0
updated_at        0
deleted_at        4
dtype: int64

admins.csv:
id            0
name          0
email         0
password      0
role          0
image_url     0
created_at    0
updated_at    0
deleted_at    5
dtype: int64

article_categories.csv:
id                       0
article_id               0
waste_category_id        0
content_category_id      0
created_at               0
updated_at               0
deleted_at             104
dtype: int64

article_comments.csv:
id               0
user_id          0
article_id       0
comment         

## Mengatasi missing values

### Table about_us_images

In [80]:
# Mengisi missing values pada kolom 'name' dengan 'Unknown'
df_about_us_images['name'] = df_about_us_images['name'].fillna('Unknown')

df_about_us_images.to_csv('staging_area/about_us_images.csv', index=False)

df_about_us_images.head()

,id,about_us_id,name,image_url,created_at,updated_at,deleted_at
0,ABSI01,ABS01,Unknown,https://res.cloudinary.com/dlbbsdd3a/image/upl...,2024-06-15 13:03:33.756,2024-06-15 13:03:33.756,NaT
1,ABSI02,ABS05,Unknown,https://res.cloudinary.com/dlbbsdd3a/image/upl...,2024-06-15 13:03:33.760,2024-06-15 13:03:33.760,NaT
2,ABSI03,ABS05,Unknown,https://res.cloudinary.com/dlbbsdd3a/image/upl...,2024-06-15 13:03:33.766,2024-06-15 13:03:33.766,NaT
3,ABSI04,ABS06,Hadyan Alhafizh,https://res.cloudinary.com/dlbbsdd3a/image/upl...,2024-06-15 13:03:33.770,2024-06-15 13:03:33.770,NaT
4,ABSI05,ABS06,Leonita Puteri Kurniawan,https://res.cloudinary.com/dlbbsdd3a/image/upl...,2024-06-15 13:03:33.774,2024-06-15 13:03:33.774,NaT


In [81]:
df_about_us_images.isnull().sum()

id              0
about_us_id     0
name            0
image_url       0
created_at      0
updated_at      0
deleted_at     26
dtype: int64

### Table article_sections

In [82]:
# Mengisi missing values pada kolom 'name' dengan 'Unknown'
df_article_sections['title'] = df_article_sections['title'].fillna('Unknown')

# Fungsi untuk menghitung jumlah kata dalam sebuah teks
def count_words(text):
    words = text.split()
    return len(words)

# Menghitung jumlah kata dalam kolom 'title'
df_article_sections['title_word_count'] = df_article_sections['title'].apply(count_words)

# Membuat mask/filter untuk baris-baris yang memiliki kurang dari 3 kata dalam kolom 'title'
mask = df_article_sections['title_word_count'] < 4

# Menghapus baris-baris yang memenuhi kondisi mask/filter
df_article_sections = df_article_sections[~mask]

# Menghapus kolom bantuan yang sudah tidak diperlukan lagi
df_article_sections = df_article_sections.drop('title_word_count', axis=1)

# Menyimpan DataFrame yang sudah difilter ke dalam file CSV baru
df_article_sections.to_csv('staging_area/article_sections.csv', index=False)

df_article_sections.head()

,id,article_id,title,description,image_url,created_at,updated_at,deleted_at
0,1,ART0001,First heavy sleep some she they.,Gifted man where. Today racism it.,https://picsum.photos/640/480,2024-01-28 14:42:26,2024-06-15 13:03:35.696,NaT
1,2,ART0001,She whose whose which am which.,Tighten we unlock. Everyone gang great.,https://picsum.photos/640/480,2024-04-21 08:14:39,2024-06-15 13:03:35.700,NaT
2,3,ART0002,Whose of into all scarcely her.,Themselves someone anyone. Whomever child heav...,https://picsum.photos/640/480,2024-01-19 04:11:55,2024-06-15 13:03:35.703,NaT
3,4,ART0002,Crawl his to besides next that.,That ring were. Up stay dangerous.,https://picsum.photos/640/480,2024-06-13 07:28:32,2024-06-15 13:03:35.708,NaT
4,5,ART0002,Within Welsh why what instance hug.,But left when. Sometimes honestly boy.,https://picsum.photos/640/480,2024-05-17 19:37:46,2024-06-15 13:03:35.713,NaT


In [83]:
df_article_sections.isnull().sum()

id               0
article_id       0
title            0
description      0
image_url        0
created_at       0
updated_at       0
deleted_at     168
dtype: int64

### Table articles

In [84]:
# Mengisi missing values pada kolom 'name' dengan 'Unknown'
df_articles['title'] = df_articles['title'].fillna('Unknown')

# Fungsi untuk menghitung jumlah kata dalam sebuah teks
def count_words(text):
    words = text.split()
    return len(words)

# Menghitung jumlah kata dalam kolom 'title'
df_articles['title_word_count'] = df_articles['title'].apply(count_words)

# Membuat mask/filter untuk baris-baris yang memiliki kurang dari 3 kata dalam kolom 'title'
mask = df_articles['title_word_count'] < 4

# Menghapus baris-baris yang memenuhi kondisi mask/filter
df_articles = df_articles[~mask]

# Menghapus kolom bantuan yang sudah tidak diperlukan lagi
df_articles = df_articles.drop('title_word_count', axis=1)

# Menyimpan DataFrame yang sudah difilter ke dalam file CSV baru
df_articles.to_csv('staging_area/articles.csv', index=False)

df_articles.head()

,id,title,description,thumbnail_url,author_id,created_at,updated_at,deleted_at
0,ART0001,Equipment panda little her board disturbed.,Out guilt nightly. This tonight greedily.,https://picsum.photos/640/480,AD0001,2024-01-17 04:11:43,2024-06-15 13:03:35.485,NaT
1,ART0002,Luxury any edify kid why his.,What aha team. Such when neither.,https://picsum.photos/640/480,AD0001,2024-05-09 20:22:51,2024-06-15 13:03:35.493,NaT
2,ART0003,Rainbow marry his edify lastly soak.,At in a. Along those poverty.,https://picsum.photos/640/480,AD0001,2024-03-22 15:53:55,2024-06-15 13:03:35.497,NaT
3,ART0004,That from game daily fade herself.,Am calm abundant. She catalog those.,https://picsum.photos/640/480,AD0001,2024-02-25 11:41:00,2024-06-15 13:03:35.501,NaT
4,ART0005,Instance live positively exactly it it.,Never what rice. These fast mercy.,https://picsum.photos/640/480,AD0001,2024-04-06 03:12:06,2024-06-15 13:03:35.505,NaT


In [85]:
df_articles.isnull().sum()

id                0
title             0
description       0
thumbnail_url     0
author_id         0
created_at        0
updated_at        0
deleted_at       49
dtype: int64

### Table reports

In [86]:
# Mengisi missing values dengan nilai default
df_reports['reason'] = df_reports['reason'].fillna('No reason provided')

# Mengisi missing values pada kolom 'name' dengan 'Unknown'
df_reports['title'] = df_reports['title'].fillna('Unknown')

# Fungsi untuk menghitung jumlah kata dalam sebuah teks
def count_words(text):
    words = text.split()
    return len(words)

# Menghitung jumlah kata dalam kolom 'title'
df_reports['title_word_count'] = df_reports['title'].apply(count_words)

# Membuat mask/filter untuk baris-baris yang memiliki kurang dari 3 kata dalam kolom 'title'
mask = df_reports['title_word_count'] < 4

# Menghapus baris-baris yang memenuhi kondisi mask/filter
df_reports = df_reports[~mask]

# Menghapus kolom bantuan yang sudah tidak diperlukan lagi
df_reports = df_reports.drop('title_word_count', axis=1)

# Menyimpan DataFrame yang sudah difilter ke dalam file CSV baru
df_reports.to_csv('staging_area/reports.csv', index=False)

df_reports.head()

,id,author_id,report_type,title,description,waste_type,latitude,longitude,address,city,province,status,reason,created_at,updated_at,deleted_at
0,RPT0001,USR0007,rubbish,Tender now everything had quietly recently.,These monthly e.g.. Pack each ever.,sampah kering,88.244187,0.302420,Jalan Slamet Riyadi,Solo,Jawa Tengah,approve,No reason provided,2024-04-24 11:03:57,2024-06-19 16:51:44.096,NaT
1,RPT0002,USR0027,rubbish,Theirs murder respond you smile doubtfully.,Of already me. Block hurriedly man.,"sampah basah, sampah kering",61.387277,-168.170093,Jalan Jendral Sudirman,Jakarta,DKI Jakarta,reject,test,2024-01-17 11:27:05,2024-06-19 16:51:44.228,NaT
2,RPT0003,USR0036,littering,Depending your occasionally sit he party.,Moment occasionally honesty. Badly club chair.,organik,-9.808979,16.569601,Jalan Pahlawan,Denpasar,Bali,reject,Laporan Tidak Lengkap,2024-06-23 16:11:34,2024-06-19 16:36:23.749,NaT
3,RPT0004,USR0036,littering,Where there one mob aha out.,Amused unusual bale. However these that.,berbahaya,83.771301,-128.691805,Jalan Riau,Bandung,Jawa Barat,reject,Could her its conclude as.,2024-06-24 21:06:40,2024-06-15 13:03:36.990,NaT
4,RPT0005,USR0015,rubbish,Whom who return cast finally anything.,Omen these down. Consequently flick where.,sampah basah,8.424071,154.131695,Jalan Imam Bonjol,Semarang,Jawa Tengah,reject,Foto kurang jelas,2024-04-21 12:27:43,2024-06-16 06:47:44.836,NaT


In [87]:
df_reports.isnull().sum()

id               0
author_id        0
report_type      0
title            0
description      0
waste_type       0
latitude         0
longitude        0
address          0
city             0
province         0
status           0
reason           0
created_at       0
updated_at       0
deleted_at     100
dtype: int64

### Table user_task_challenges

In [88]:
# Mengisi missing values pada kolom 'description_image' dengan 'No description provided'
df_user_task_challenges['description_image'] = df_user_task_challenges['description_image'].fillna('No description provided')

# Mengisi missing values pada kolom 'reason' dengan 'No reason provided'
df_user_task_challenges['reason'] = df_user_task_challenges['reason'].fillna('No reason provided')

df_user_task_challenges.to_csv('staging_area/user_task_challenges.csv', index=False)

df_user_task_challenges.head()

,id,user_id,task_challenge_id,status_progress,status_accept,description_image,point,reason,accepted_at,created_at,updated_at,deleted_at
0,UT0001,USR0001,TM0018,in_progress,need_rivew,No description provided,0,No reason provided,2024-06-15 14:07:14,2024-06-15 14:07:13.863,2024-06-15 14:07:13.863,NaT
1,UT0002,USR0001,TM0010,in_progress,need_rivew,No description provided,0,No reason provided,2024-06-15 16:53:09,2024-06-15 16:53:08.910,2024-06-15 16:53:08.910,NaT
2,UT0003,USR0001,TM0017,in_progress,need_rivew,No description provided,0,No reason provided,2024-06-15 16:58:23,2024-06-15 16:58:23.310,2024-06-15 16:58:23.310,NaT
3,UT0004,USR0001,TM0015,in_progress,need_rivew,No description provided,0,No reason provided,2024-06-16 01:45:44,2024-06-16 01:45:43.922,2024-06-16 01:45:43.922,NaT
4,UT0005,USR0034,TM0009,done,accept,Test,588,No reason provided,2024-06-18 15:31:19,2024-06-18 13:42:58.948,2024-06-18 15:31:19.353,NaT


In [89]:
df_user_task_challenges.isnull().sum()

id                    0
user_id               0
task_challenge_id     0
status_progress       0
status_accept         0
description_image     0
point                 0
reason                0
accepted_at           0
created_at            0
updated_at            0
deleted_at           31
dtype: int64

### Table task_steps

In [90]:
# Fungsi untuk menghitung jumlah kata dalam sebuah teks
def count_words(text):
    words = text.split()
    return len(words)

# Menghitung jumlah kata dalam kolom 'description'
df_task_steps['description_word_count'] = df_task_steps['description'].apply(count_words)

# Membuat mask/filter untuk baris-baris yang memiliki kurang dari 3 kata dalam kolom 'description'
mask = df_task_steps['description_word_count'] < 3

# Menghapus baris-baris yang memenuhi kondisi mask/filter
df_task_steps = df_task_steps[~mask]

# Menghapus kolom bantuan yang sudah tidak diperlukan lagi
df_task_steps = df_task_steps.drop('description_word_count', axis=1)

# Menyimpan DataFrame yang sudah difilter ke dalam file CSV baru
df_task_steps.to_csv('staging_area/task_steps.csv', index=False)

df_task_steps.head()

,id,task_challenge_id,title,description,created_at,updated_at,deleted_at
0,1,TM0001,Step 1,These why you. Band sorrow theirs.,2024-01-23 07:51:11,2024-06-15 13:03:34.221,NaT
1,2,TM0001,Step 2,In off him. Besides that his.,2024-01-06 15:19:29,2024-06-15 13:03:34.225,NaT
2,3,TM0001,Step 3,Such in would. This should indeed.,2024-02-17 03:44:13,2024-06-15 13:03:34.230,NaT
3,4,TM0001,Step 4,Double their alternatively. Without ours below.,2024-02-20 05:12:57,2024-06-15 13:03:34.234,NaT
4,5,TM0002,Step 1,From summation congregation. Have e.g. even.,2024-04-12 04:22:38,2024-06-15 13:03:34.239,NaT


### Table task_challenges

In [91]:
# Fungsi untuk menghitung jumlah kata dalam sebuah teks
def count_words(text):
    words = text.split()
    return len(words)

# Menghitung jumlah kata dalam kolom 'description'
df_task_challenges['description_word_count'] = df_task_challenges['description'].apply(count_words)

# Membuat mask/filter untuk baris-baris yang memiliki kurang dari 3 kata dalam kolom 'description'
mask = df_task_challenges['description_word_count'] < 3

# Menghapus baris-baris yang memenuhi kondisi mask/filter
df_task_challenges = df_task_challenges[~mask]

# Menghapus kolom bantuan yang sudah tidak diperlukan lagi
df_task_challenges = df_task_challenges.drop('description_word_count', axis=1)

# Menyimpan DataFrame yang sudah difilter ke dalam file CSV baru
df_task_challenges.to_csv('staging_area/task_challenges.csv', index=False)

df_task_challenges.head()

,id,title,description,thumbnail,start_date,end_date,point,status,admin_id,created_at,updated_at,deleted_at
0,TM0001,Us philosophy crowd any utterly about.,Therefore have whom. Below furthermore every.,https://picsum.photos/640/480,2024-03-20 14:40:03,2024-03-27 14:40:03,2277,0,AD0001,2024-02-24 17:26:52,2024-06-16 00:00:00.008,NaT
1,TM0002,Him head sneeze one yourselves life.,Brilliance firstly occasionally. Who occasiona...,https://picsum.photos/640/480,2024-05-08 01:11:00,2024-05-11 01:11:00,1891,0,AD0001,2024-03-19 14:39:11,2024-06-16 00:00:00.008,NaT
2,TM0003,Accordingly along crew everything nightly due.,Will then a. In team everyone.,https://picsum.photos/640/480,2024-01-03 10:16:00,2024-01-08 10:16:00,966,0,AD0001,2024-01-01 03:15:46,2024-06-16 00:00:00.008,NaT
3,TM0004,Yourselves child mob break those energetic.,There moreover chase. Lots those bravo.,https://picsum.photos/640/480,2024-01-13 14:28:14,2024-01-16 14:28:14,955,0,AD0001,2024-01-09 22:50:31,2024-06-16 00:00:00.008,NaT
4,TM0005,Summation dangerous purely mine once ski.,So far just. May so these.,https://picsum.photos/640/480,2024-03-27 19:13:24,2024-04-02 19:13:24,1616,0,AD0001,2024-02-13 01:44:44,2024-06-16 00:00:00.008,NaT


### Table users

In [92]:
# Mengganti nilai yang hilang (NaN) dengan tanda '-'
df_users['gender'] = df_users['gender'].fillna('-')
df_users['birth_date'] = df_users['birth_date'].fillna('-')
df_users['address'] = df_users['address'].fillna('-')
df_users['picture_url'] = df_users['picture_url'].fillna('-')

df_users.to_csv('staging_area/users.csv', index=False)

df_users.head()

,id,name,email,password,point,gender,birth_date,address,picture_url,otp,is_verified,badge,created_at,updated_at,deleted_at
0,USR0001,Clinton Swaniawski,joshuahmante@kassulke.com,$2a$12$5KC.m.K50I2uW4xVUDxQyOrLUraC1kbr88y2/Le...,386284,perempuan,1998-06-20,"Jalan Malioboro, Yogyakarta, Indonesia",https://picsum.photos/200/200,570177,1,platinum,2024-03-24 06:25:37,2024-06-15 13:03:32.991,2024-06-16 17:35:04.506
1,USR0002,Joelle Mann,ollieflatley@kunze.biz,$2a$12$5KC.m.K50I2uW4xVUDxQyOrLUraC1kbr88y2/Le...,320221,perempuan,2003-05-16,"Jalan Tunjungan, Surabaya, Indonesia",https://picsum.photos/200/200,137299,1,platinum,2024-06-01 18:01:58,2024-06-15 13:03:32.995,NaT
2,USR0003,Austen Pfeffer,brodyhoeger@halvorson.biz,$2a$12$5KC.m.K50I2uW4xVUDxQyOrLUraC1kbr88y2/Le...,47791,laki-laki,1976-04-17,"Jalan Gatot Subroto, Jakarta, Indonesia",https://res.cloudinary.com/dlbbsdd3a/image/upl...,556740,1,classic,2024-06-17 04:21:57,2024-06-19 09:59:08.101,NaT
3,USR0004,Joanie Turcotte,metapredovic@daniel.org,$2a$12$5KC.m.K50I2uW4xVUDxQyOrLUraC1kbr88y2/Le...,164319,perempuan,1994-12-02,"Jalan Panglima Polim, Jakarta, Indonesia",https://res.cloudinary.com/dlbbsdd3a/image/upl...,419910,1,gold,2024-06-02 15:56:00,2024-06-19 14:40:22.949,NaT
4,USR0005,Toney Kirlin,lorenhermann@gorczany.info,$2a$12$5KC.m.K50I2uW4xVUDxQyOrLUraC1kbr88y2/Le...,46040,laki-laki,1976-05-27,"Jalan Panglima Polim, Jakarta, Indonesia",https://picsum.photos/200/200,440526,1,classic,2024-02-05 07:36:27,2024-06-15 13:03:33.010,NaT


In [93]:
df_users.isnull().sum()

id              0
name            0
email           0
password        0
point           0
gender          0
birth_date      0
address         0
picture_url     0
otp             0
is_verified     0
badge           0
created_at      0
updated_at      0
deleted_at     55
dtype: int64

## Mengecek dan mengatasi duplikasi data

In [94]:
data_files = [
    ("about_us_images.csv", df_about_us_images),
    ("about_us.csv", df_about_us),
    ("achievements.csv", df_achievements),
    ("admins.csv", df_admins),
    ("article_categories.csv", df_article_categories),
    ("article_comments.csv", df_article_comments),
    ("article_sections.csv", df_article_sections),
    ("articles.csv", df_articles),
    ("comments.csv", df_comments),
    ("content_categories.csv", df_content_categories),
    ("custom_data.csv", df_custom_data),
    ("faqs.csv", df_faqs),
    ("report_images.csv", df_report_images),
    ("report_waste_materials.csv", df_report_waste_materials),
    ("reports.csv", df_reports),
    ("task_challenges.csv", df_task_challenges),
    ("task_steps.csv", df_task_steps),
    ("user_task_challenges.csv", df_user_task_challenges),
    ("user_task_images.csv", df_user_task_images),
    ("user_task_steps.csv", df_user_task_steps),
    ("users.csv", df_users),
    ("video_categories.csv", df_video_categories),
    ("videos.csv", df_videos),
    ("waste_categories.csv", df_waste_categories),
    ("waste_materials.csv", df_waste_materials),
]

# Direktori tempat menyimpan file-file yang diperbarui
staging_area = 'staging_area'

# Iterasi melalui setiap file dan DataFrame untuk menangani duplikat
for file_name, dataset in data_files:
    # Mendeteksi dan menangani duplikat
    initial_rows = dataset.shape[0]
    dataset.drop_duplicates(inplace=True)
    new_rows = dataset.shape[0]
    
    # Membuat path lengkap untuk menyimpan file CSV yang diperbarui di dalam staging_area
    file_path = os.path.join(staging_area, file_name)
    
    # Menyimpan DataFrame yang sudah diperbarui ke dalam file CSV baru
    dataset.to_csv(file_path, index=False)
    
    # Menampilkan informasi tentang duplikat
    if new_rows < initial_rows:
        print(f"\nDuplikasi data di {file_name} ditemukan dan dihapus.")
        print(f"Jumlah baris sebelum: {initial_rows}, setelah: {new_rows}.")
    else:
        print(f"\nTidak ada duplikasi data di {file_name}.")


Tidak ada duplikasi data di about_us_images.csv.

Tidak ada duplikasi data di about_us.csv.

Tidak ada duplikasi data di achievements.csv.

Tidak ada duplikasi data di admins.csv.

Tidak ada duplikasi data di article_categories.csv.

Tidak ada duplikasi data di article_comments.csv.

Tidak ada duplikasi data di article_sections.csv.

Tidak ada duplikasi data di articles.csv.

Tidak ada duplikasi data di comments.csv.

Tidak ada duplikasi data di content_categories.csv.

Tidak ada duplikasi data di custom_data.csv.

Tidak ada duplikasi data di faqs.csv.

Tidak ada duplikasi data di report_images.csv.

Tidak ada duplikasi data di report_waste_materials.csv.

Tidak ada duplikasi data di reports.csv.

Tidak ada duplikasi data di task_challenges.csv.

Tidak ada duplikasi data di task_steps.csv.

Tidak ada duplikasi data di user_task_challenges.csv.

Tidak ada duplikasi data di user_task_images.csv.

Tidak ada duplikasi data di user_task_steps.csv.

Tidak ada duplikasi data di users.csv.

T

## Menentukan data untuk data warehouse

### Tabel fact_reporting

In [95]:
reports = pd.read_csv('staging_area/reports.csv')
report_waste_materials = pd.read_csv('staging_area/report_waste_materials.csv')

In [96]:
df_fact_reporting = (
    reports
    .merge(report_waste_materials, left_on='id', right_on='report_id')
    [['id_x', 'id_y', 'author_id', 'report_type']]
    .rename(columns={'id_x' : 'id', 'id_y' : 'report_waste_materials', 'author_id' : 'user_id'})
)

In [97]:
df_fact_reporting.to_csv('data_warehouse/fact_reporting.csv', index=False)
df_fact_reporting

,id,report_waste_materials,user_id,report_type
0,RPT0001,29372d78-a6b9-480f-82c1-27485b2ffc78,USR0007,rubbish
1,RPT0002,6286d5fa-5384-4cbe-9f92-ad00f43e567b,USR0027,rubbish
2,RPT0002,ffb4e8ef-f3a3-4921-89e5-d2d1fde7dcd8,USR0027,rubbish
3,RPT0005,08eb7bd0-66dc-4665-b209-3c2adc2f37b7,USR0015,rubbish
4,RPT0005,0e57e4ca-377f-4a48-936e-be71c0fd17b4,USR0015,rubbish
...,...,...,...,...
112,RPT0096,41ed9c13-a265-43de-996a-c18371ad1dd5,USR0011,rubbish
113,RPT0096,b7a38649-1c8b-4ce4-9a3c-ff2c60581014,USR0011,rubbish
114,RPT0096,fb3a9ef1-6002-4eac-8aff-1c328de98dd1,USR0011,rubbish
115,RPT0098,3ceff3e2-ec24-4476-aa9b-efd9f73de16a,USR0030,rubbish


### Table fact_challange

In [98]:
task_challenges = pd.read_csv('staging_area/task_challenges.csv')
user_task_challenges = pd.read_csv('staging_area/user_task_challenges.csv')

In [99]:
df_fact_challange = (
    task_challenges
    .merge(user_task_challenges, left_on='id', right_on='task_challenge_id')
    [['id_x', 'id_y', 'user_id', 'status_accept']]
    .rename(columns={'id_x' : 'id', 'id_y' : 'user_task_challenges'})
)

In [100]:
df_fact_challange.to_csv('data_warehouse/fact_challange.csv', index=False)
df_fact_challange

,id,user_task_challenges,user_id,status_accept
0,TM0007,UT0026,USR0006,need_rivew
1,TM0008,UT0006,USR0034,reject
2,TM0008,UT0009,USR0002,need_rivew
3,TM0008,UT0030,USR0007,need_rivew
4,TM0008,UT0031,USR0004,need_rivew
5,TM0009,UT0005,USR0034,accept
6,TM0009,UT0010,USR0002,need_rivew
7,TM0009,UT0015,USR0003,need_rivew
8,TM0009,UT0024,USR0005,need_rivew
9,TM0009,UT0027,USR0006,need_rivew


### Table fact_videos

In [101]:
df_videos = pd.read_csv('staging_area/videos.csv')
df_comments = pd.read_csv('staging_area/comments.csv')

In [102]:
df_fact_videos_comment = (
    df_videos
    .merge(df_comments, left_on='id', right_on='video_id')
    [['id_x', 'user_id', 'id_y', 'comment']]
    .rename(columns={'id_x' : 'id', 'id_y' : 'videos_comments_id'})
)

In [103]:
df_fact_videos_comment.to_csv('data_warehouse/fact_videos_comment.csv', index=False)
df_fact_videos_comment

,id,user_id,videos_comments_id,comment
0,1,USR0043,1501,Have they next those constantly.
1,1,USR0018,1502,Which trip myself that some.
2,1,USR0023,1503,Whose whomever these mine why.
3,1,USR0039,1504,What been yearly could number.
4,1,USR0046,1505,Other firstly of their horror.
...,...,...,...,...
1497,50,USR0005,2996,Pack occasionally why before off.
1498,50,USR0035,2997,Were some somebody whom Barcelonian.
1499,50,USR0023,2998,Her perfectly place off hers.
1500,50,USR0038,2999,Aircraft onto that then your.


### Table fact articles

In [104]:
df_articles = pd.read_csv('staging_area/articles.csv')
df_article_comments = pd.read_csv('staging_area/article_comments.csv')

In [105]:
df_fact_articles_comment = (
    df_articles
    .merge(df_article_comments, left_on='id', right_on='article_id')
    [['id_x', 'user_id', 'id_y', 'comment']]
    .rename(columns={'id_x' : 'id', 'id_y' : 'article_comments_id'})
)

In [106]:
df_fact_articles_comment.to_csv('data_warehouse/fact_articles_comment.csv', index=False)
df_fact_articles_comment

,id,user_id,article_comments_id,comment
0,ART0001,USR0006,1,Envious condemned Nepalese the utterly.
1,ART0001,USR0013,2,Would election way Dutch none.
2,ART0001,USR0015,3,Down little without myself time.
3,ART0001,USR0042,4,Myself troupe ouch work therefore.
4,ART0001,USR0019,5,Is this sunshine as any.
...,...,...,...,...
1500,ART0050,USR0010,1496,Dance Afghan wait reel their.
1501,ART0050,USR0007,1497,Here fact there luxuty quarterly.
1502,ART0050,USR0040,1498,Whom lead toes as this.
1503,ART0050,USR0020,1499,To what regularly all Diabolical.


### Table users

In [107]:
df_users = pd.read_csv('staging_area/users.csv')

dim_users_columns = ['id', 'name', 'email', 'point', 'gender', 
                    'birth_date', 'address', 'badge', 
                    'created_at', 'updated_at', 'deleted_at']

df_dim_users = df_users[dim_users_columns]

df_dim_users.to_csv('data_warehouse/users.csv', index=False)

df_dim_users.head()

,id,name,email,point,gender,birth_date,address,badge,created_at,updated_at,deleted_at
0,USR0001,Clinton Swaniawski,joshuahmante@kassulke.com,386284,perempuan,1998-06-20,"Jalan Malioboro, Yogyakarta, Indonesia",platinum,2024-03-24 06:25:37.000,2024-06-15 13:03:32.991,2024-06-16 17:35:04.506
1,USR0002,Joelle Mann,ollieflatley@kunze.biz,320221,perempuan,2003-05-16,"Jalan Tunjungan, Surabaya, Indonesia",platinum,2024-06-01 18:01:58.000,2024-06-15 13:03:32.995,NaN
2,USR0003,Austen Pfeffer,brodyhoeger@halvorson.biz,47791,laki-laki,1976-04-17,"Jalan Gatot Subroto, Jakarta, Indonesia",classic,2024-06-17 04:21:57.000,2024-06-19 09:59:08.101,NaN
3,USR0004,Joanie Turcotte,metapredovic@daniel.org,164319,perempuan,1994-12-02,"Jalan Panglima Polim, Jakarta, Indonesia",gold,2024-06-02 15:56:00.000,2024-06-19 14:40:22.949,NaN
4,USR0005,Toney Kirlin,lorenhermann@gorczany.info,46040,laki-laki,1976-05-27,"Jalan Panglima Polim, Jakarta, Indonesia",classic,2024-02-05 07:36:27.000,2024-06-15 13:03:33.010,NaN


### Table reports

In [108]:
df_reports = pd.read_csv('staging_area/reports.csv')

dim_reports_columns = ['id', 'report_type', 'title', 'description', 'waste_type',
                       'latitude','longitude', 'address', 'city', 'province',
                       'status', 'reason', 'created_at', 'updated_at', 'deleted_at']

df_dim_reports = df_reports[dim_reports_columns]

df_dim_reports.to_csv('data_warehouse/reports.csv', index=False)

df_dim_reports.head()

,id,report_type,title,description,waste_type,latitude,longitude,address,city,province,status,reason,created_at,updated_at,deleted_at
0,RPT0001,rubbish,Tender now everything had quietly recently.,These monthly e.g.. Pack each ever.,sampah kering,88.244187,0.302420,Jalan Slamet Riyadi,Solo,Jawa Tengah,approve,No reason provided,2024-04-24 11:03:57,2024-06-19 16:51:44.096,NaN
1,RPT0002,rubbish,Theirs murder respond you smile doubtfully.,Of already me. Block hurriedly man.,"sampah basah, sampah kering",61.387277,-168.170093,Jalan Jendral Sudirman,Jakarta,DKI Jakarta,reject,test,2024-01-17 11:27:05,2024-06-19 16:51:44.228,NaN
2,RPT0003,littering,Depending your occasionally sit he party.,Moment occasionally honesty. Badly club chair.,organik,-9.808979,16.569601,Jalan Pahlawan,Denpasar,Bali,reject,Laporan Tidak Lengkap,2024-06-23 16:11:34,2024-06-19 16:36:23.749,NaN
3,RPT0004,littering,Where there one mob aha out.,Amused unusual bale. However these that.,berbahaya,83.771301,-128.691805,Jalan Riau,Bandung,Jawa Barat,reject,Could her its conclude as.,2024-06-24 21:06:40,2024-06-15 13:03:36.990,NaN
4,RPT0005,rubbish,Whom who return cast finally anything.,Omen these down. Consequently flick where.,sampah basah,8.424071,154.131695,Jalan Imam Bonjol,Semarang,Jawa Tengah,reject,Foto kurang jelas,2024-04-21 12:27:43,2024-06-16 06:47:44.836,NaN


### Table report_waste_materials

In [109]:
df_report_waste_materials = pd.read_csv('staging_area/report_waste_materials.csv')

dim_report_waste_materials_columns = ['id', 'waste_material', 'created_at', 'updated_at', 'deleted_at']

df_dim_report_waste_materials = df_report_waste_materials[dim_report_waste_materials_columns]

df_dim_report_waste_materials.to_csv('data_warehouse/report_waste_materials.csv', index=False)

df_dim_report_waste_materials.head()

,id,waste_material,created_at,updated_at,deleted_at
0,028b3c7e-fb45-417a-8f67-103d4e600a7f,limbah beracun,2024-06-15 13:03:37.575,2024-06-15 13:03:37.575,NaN
1,04ad45df-cda8-4319-a5dd-223872e68f07,kaca,2024-06-15 13:03:37.503,2024-06-15 13:03:37.503,NaN
2,0594eac5-c6bb-43c9-86ee-7fd19c833b33,kertas,2024-06-15 13:03:37.688,2024-06-15 13:03:37.688,NaN
3,06cc7d48-d03b-4d96-aa16-74496f30af5f,plastik,2024-06-15 13:03:37.845,2024-06-15 13:03:37.845,NaN
4,07cca3b6-6d2c-431b-ae1b-32addae6ca36,kayu,2024-06-15 13:03:37.730,2024-06-15 13:03:37.730,NaN


### Table user_task_challenges

In [110]:
df_user_task_challenges = pd.read_csv('staging_area/user_task_challenges.csv')

dim_user_task_challenges_columns = ['id', 'status_progress', 'status_accept', 'point',
                                    'reason', 'accepted_at', 'created_at', 'updated_at', 'deleted_at']

df_dim_user_task_challenges = df_user_task_challenges[dim_user_task_challenges_columns]

df_dim_user_task_challenges.to_csv('data_warehouse/user_task_challenges.csv', index=False)

df_dim_user_task_challenges.head()

,id,status_progress,status_accept,point,reason,accepted_at,created_at,updated_at,deleted_at
0,UT0001,in_progress,need_rivew,0,No reason provided,2024-06-15 14:07:14,2024-06-15 14:07:13.863,2024-06-15 14:07:13.863,NaN
1,UT0002,in_progress,need_rivew,0,No reason provided,2024-06-15 16:53:09,2024-06-15 16:53:08.910,2024-06-15 16:53:08.910,NaN
2,UT0003,in_progress,need_rivew,0,No reason provided,2024-06-15 16:58:23,2024-06-15 16:58:23.310,2024-06-15 16:58:23.310,NaN
3,UT0004,in_progress,need_rivew,0,No reason provided,2024-06-16 01:45:44,2024-06-16 01:45:43.922,2024-06-16 01:45:43.922,NaN
4,UT0005,done,accept,588,No reason provided,2024-06-18 15:31:19,2024-06-18 13:42:58.948,2024-06-18 15:31:19.353,NaN


### Table task_challenges

In [111]:
df_task_challenges = pd.read_csv('staging_area/task_challenges.csv')

dim_task_challenges_columns = ['id', 'title', 'description', 'start_date', 'end_date', 'point', 'status', 'created_at', 'updated_at', 'deleted_at']

df_dim_task_challenges = df_task_challenges[dim_task_challenges_columns]

df_dim_task_challenges.to_csv('data_warehouse/task_challenges.csv', index=False)

df_dim_task_challenges.head()

,id,title,description,start_date,end_date,point,status,created_at,updated_at,deleted_at
0,TM0001,Us philosophy crowd any utterly about.,Therefore have whom. Below furthermore every.,2024-03-20 14:40:03,2024-03-27 14:40:03,2277,0,2024-02-24 17:26:52.000,2024-06-16 00:00:00.008,NaN
1,TM0002,Him head sneeze one yourselves life.,Brilliance firstly occasionally. Who occasiona...,2024-05-08 01:11:00,2024-05-11 01:11:00,1891,0,2024-03-19 14:39:11.000,2024-06-16 00:00:00.008,NaN
2,TM0003,Accordingly along crew everything nightly due.,Will then a. In team everyone.,2024-01-03 10:16:00,2024-01-08 10:16:00,966,0,2024-01-01 03:15:46.000,2024-06-16 00:00:00.008,NaN
3,TM0004,Yourselves child mob break those energetic.,There moreover chase. Lots those bravo.,2024-01-13 14:28:14,2024-01-16 14:28:14,955,0,2024-01-09 22:50:31.000,2024-06-16 00:00:00.008,NaN
4,TM0005,Summation dangerous purely mine once ski.,So far just. May so these.,2024-03-27 19:13:24,2024-04-02 19:13:24,1616,0,2024-02-13 01:44:44.000,2024-06-16 00:00:00.008,NaN


### Table videos

In [112]:
df_videos = pd.read_csv('staging_area/videos.csv')

dim_videos_columns = ['id', 'title', 'description', 'link', 'viewer', 'created_at', 'updated_at', 'deleted_at']

df_dim_videos = df_videos[dim_videos_columns]

df_dim_videos.to_csv('data_warehouse/videos.csv', index=False)

df_dim_videos.head()

,id,title,description,link,viewer,created_at,updated_at,deleted_at
0,1,lah update,Video Description,https://www.youtube.com/watch?v=NH9yuZUrJVc,658331,2024-04-07 14:49:15.000,2024-06-16 18:00:49.986,2024-06-16 18:12:34.522
1,2,She within way trust quarterly others.,Upstairs inside for. Already party here.,https://www.youtube.com/watch?v=CGd3lgxReFE,0,2024-01-02 06:14:54.000,2024-06-15 13:03:34.664,2024-06-16 18:10:26.261
2,3,Of lovely huh off their crime.,Amused under what. Itself herself company.,https://www.youtube.com/watch?v=CGd3lgxReFE,199917,2024-04-20 07:15:07.000,2024-06-19 09:49:13.031,NaN
3,4,Shall towards anyone begin did quiver.,Page no little. Nightly had finally.,https://www.youtube.com/watch?v=CGd3lgxReFE,199917,2024-04-23 04:02:40.000,2024-06-19 09:49:12.818,NaN
4,5,Thing are lie cabinet select whoa.,In behind example. Party everything including.,https://www.youtube.com/watch?v=CGd3lgxReFE,199917,2024-01-03 11:21:37.000,2024-06-19 09:49:20.075,NaN


### Table comments

In [113]:
df_comments = pd.read_csv('staging_area/comments.csv')

dim_comments_columns = ['id', 'comment', 'created_at', 'updated_at', 'deleted_at']

df_dim_comments = df_comments[dim_comments_columns]

df_dim_comments.to_csv('data_warehouse/comments.csv', index=False)

df_dim_comments.head()

,id,comment,created_at,updated_at,deleted_at
0,1501,Have they next those constantly.,2024-04-02 06:26:39.948,2024-06-18 12:11:26.250,NaN
1,1502,Which trip myself that some.,2024-02-23 19:26:14.916,2024-06-18 12:11:26.258,NaN
2,1503,Whose whomever these mine why.,2024-05-15 16:03:19.458,2024-06-18 12:11:26.263,NaN
3,1504,What been yearly could number.,2024-03-05 11:19:29.583,2024-06-18 12:11:26.268,NaN
4,1505,Other firstly of their horror.,2024-02-06 20:36:44.365,2024-06-18 12:11:26.272,NaN


### Table articles

In [114]:
f_articles = pd.read_csv('staging_area/articles.csv')

dim_articles_columns = ['id', 'title', 'description', 'created_at', 'updated_at', 'deleted_at']

df_dim_articles = df_articles[dim_articles_columns]

df_dim_articles.to_csv('data_warehouse/articles.csv', index=False)

df_dim_articles.head()

,id,title,description,created_at,updated_at,deleted_at
0,ART0001,Equipment panda little her board disturbed.,Out guilt nightly. This tonight greedily.,2024-01-17 04:11:43.000,2024-06-15 13:03:35.485,NaN
1,ART0002,Luxury any edify kid why his.,What aha team. Such when neither.,2024-05-09 20:22:51.000,2024-06-15 13:03:35.493,NaN
2,ART0003,Rainbow marry his edify lastly soak.,At in a. Along those poverty.,2024-03-22 15:53:55.000,2024-06-15 13:03:35.497,NaN
3,ART0004,That from game daily fade herself.,Am calm abundant. She catalog those.,2024-02-25 11:41:00.000,2024-06-15 13:03:35.501,NaN
4,ART0005,Instance live positively exactly it it.,Never what rice. These fast mercy.,2024-04-06 03:12:06.000,2024-06-15 13:03:35.505,NaN


### Table article_comments

In [115]:
df_article_comments = pd.read_csv('staging_area/article_comments.csv')

dim_article_comments_columns = ['id', 'comment', 'created_at', 'updated_at', 'deleted_at']

df_dim_article_comments = df_article_comments[dim_article_comments_columns]

df_dim_article_comments.to_csv('data_warehouse/article_comments.csv', index=False)

df_dim_article_comments.head()

,id,comment,created_at,updated_at,deleted_at
0,1,Envious condemned Nepalese the utterly.,2024-05-06 04:22:34.671,2024-06-18 12:11:19.659,NaN
1,2,Would election way Dutch none.,2024-04-18 03:05:16.006,2024-06-18 12:11:19.662,NaN
2,3,Down little without myself time.,2024-03-16 12:50:49.073,2024-06-18 12:11:19.669,NaN
3,4,Myself troupe ouch work therefore.,2024-05-07 16:54:50.618,2024-06-18 12:11:19.673,NaN
4,5,Is this sunshine as any.,2024-03-09 23:30:22.960,2024-06-18 12:11:19.677,NaN


# LOAD

## Load to Google Cloud Storage

Load database back end for archive

In [116]:
load_dotenv()

# Inisialisasi kredensial dari file JSON
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.getenv('GCS')

# Inisialisasi client GCS
client = storage.Client()

# Nama bucket dan folder di GCS
bucket_name = "recything-data-archive"
folder_name = "data_database"

# Daftar file yang ingin diunggah
files_to_upload = [
    'about_us_images.csv', 'about_us.csv', 'achievements.csv', 'admins.csv',
    'article_categories.csv', 'article_comments.csv', 'article_sections.csv',
    'articles.csv', 'comments.csv', 'content_categories.csv', 'custom_data.csv',
    'faqs.csv', 'report_images.csv', 'report_waste_materials.csv', 'reports.csv',
    'task_challenges.csv', 'task_steps.csv', 'user_task_challenges.csv',
    'user_task_images.csv', 'user_task_steps.csv', 'users.csv',
    'video_categories.csv', 'videos.csv', 'waste_categories.csv', 'waste_materials.csv'
]

# Loop untuk mengunggah setiap file ke GCS
for file_name in files_to_upload:
    local_file_path = f"./data_database/{file_name}"

    # Upload file ke GCS
    blob = client.bucket(bucket_name).blob(f"{folder_name}/{file_name}")
    blob.upload_from_filename(local_file_path)

    print(f"File {file_name} berhasil diunggah ke GCS.")

File about_us_images.csv berhasil diunggah ke GCS.
File about_us.csv berhasil diunggah ke GCS.
File achievements.csv berhasil diunggah ke GCS.
File admins.csv berhasil diunggah ke GCS.
File article_categories.csv berhasil diunggah ke GCS.
File article_comments.csv berhasil diunggah ke GCS.
File article_sections.csv berhasil diunggah ke GCS.
File articles.csv berhasil diunggah ke GCS.
File comments.csv berhasil diunggah ke GCS.
File content_categories.csv berhasil diunggah ke GCS.
File custom_data.csv berhasil diunggah ke GCS.
File faqs.csv berhasil diunggah ke GCS.
File report_images.csv berhasil diunggah ke GCS.
File report_waste_materials.csv berhasil diunggah ke GCS.
File reports.csv berhasil diunggah ke GCS.
File task_challenges.csv berhasil diunggah ke GCS.
File task_steps.csv berhasil diunggah ke GCS.
File user_task_challenges.csv berhasil diunggah ke GCS.
File user_task_images.csv berhasil diunggah ke GCS.
File user_task_steps.csv berhasil diunggah ke GCS.
File users.csv berhasi

## Load to Big Query

Load data for data warehouse

In [117]:
load_dotenv()

# Path ke file key JSON
key_path = os.getenv('GOOGLE_APPLICATION_CREDENTIALS')

# Memastikan key_path dimuat dengan benar
if not key_path:
    raise ValueError("GOOGLE_APPLICATION_CREDENTIALS key path not found.")

# Set Google Cloud credentials
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = key_path

# BigQuery client
client = bigquery.Client()

# Nama dataset
dataset_name = 'recything_datawarehouse'

# Folder data
data_folder = 'data_warehouse'

# List data warehouse
csv_files = [
    'article_comments.csv', 'articles.csv', 'comments.csv', 'fact_articles_comment.csv',
    'fact_challange.csv', 'fact_reporting.csv', 'fact_videos_comment.csv', 'report_waste_materials.csv',
    'reports.csv', 'task_challenges.csv', 'user_task_challenges.csv', 'users.csv',
    'videos.csv'
]

# Fungsi untuk mengunggah file CSV ke BigQuery
def load_csv_to_bigquery(file_name):
    file_path = os.path.join(data_folder, file_name)
    table_name = file_name.replace('.csv', '')
    
    # Muat file CSV ke dalam DataFrame pandas
    df = pd.read_csv(file_path)
    
    # Tentukan ID tabel
    table_id = f"{client.project}.{dataset_name}.{table_name}"
    
    # Definisikan konfigurasi job
    job_config = bigquery.LoadJobConfig(
        write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE  # Replace table if it already exists
    )
    
    # Muat DataFrame ke tabel BigQuery
    job = client.load_table_from_dataframe(df, table_id, job_config=job_config)
    job.result()
    
    print(f"Loaded {file_name} into {table_id}")

# Memastikan dataset ada
dataset_id = f"{client.project}.{dataset_name}"
try:
    client.get_dataset(dataset_id)
    print(f"Dataset {dataset_name} already exists")
except NotFound:
    # Create a new dataset
    dataset = bigquery.Dataset(dataset_id)
    dataset = client.create_dataset(dataset, timeout=30)
    print(f"Created dataset {dataset_name}")

# Upload file ke BigQuery
for csv_file in csv_files:
    load_csv_to_bigquery(csv_file)
print("Data successfully uploaded")

Dataset recything_datawarehouse already exists
Loaded article_comments.csv into capstone-426614.recything_datawarehouse.article_comments
Loaded articles.csv into capstone-426614.recything_datawarehouse.articles
Loaded comments.csv into capstone-426614.recything_datawarehouse.comments
Loaded fact_articles_comment.csv into capstone-426614.recything_datawarehouse.fact_articles_comment
Loaded fact_challange.csv into capstone-426614.recything_datawarehouse.fact_challange
Loaded fact_reporting.csv into capstone-426614.recything_datawarehouse.fact_reporting
Loaded fact_videos_comment.csv into capstone-426614.recything_datawarehouse.fact_videos_comment
Loaded report_waste_materials.csv into capstone-426614.recything_datawarehouse.report_waste_materials
Loaded reports.csv into capstone-426614.recything_datawarehouse.reports
Loaded task_challenges.csv into capstone-426614.recything_datawarehouse.task_challenges
Loaded user_task_challenges.csv into capstone-426614.recything_datawarehouse.user_tas